In [ ]:
# -*- coding: utf-8 -*-
import sqlite3 # 导入 SQLite 数据库模块，用于存储文档元数据
import os      # 导入操作系统模块，用于文件路径操作和检查文件是否存在
import numpy as np # 导入 NumPy 库，用于高效处理数值数组，特别是向量操作
from typing import List, Dict, Union, Optional, Tuple, Dict # 增加了类型提示，提高代码可读性和健壮性
import faiss   # 导入 Faiss 库，用于高效的向量相似性搜索和索引
               # 需要先安装 faiss-cpu 或 faiss-gpu: pip install faiss-cpu (或 faiss-gpu)
from transformers import CLIPProcessor, CLIPModel # 导入 Hugging Face Transformers 库中的 CLIP 模型和处理器
                                                 # CLIP (Contrastive Language–Image Pre-training) 是一种强大的多模态模型
                                                 # 需要先安装: pip install transformers torch pillow
from PIL import Image # 导入 Pillow 库 (PIL fork)，用于图像文件的加载和基本处理
import torch   # 导入 PyTorch 库，Transformers 库基于 PyTorch 构建
import zhipuai # 导入 ZhipuAI 客户端库，用于与智谱 AI 的大语言模型 API 进行交互
               # 需要先安装: pip install zhipuai
# import base64 # (注释掉) 如果需要将图像编码为 base64 字符串传输，可以取消注释
# import io     # (注释掉) 如果需要将图像数据在内存中处理，可以取消注释

import json    # 导入 JSON 库，用于加载存储在 .json 文件中的初始文档数据
import time    # 导入时间库，用于在流程中添加暂停，方便观察输出或避免 API 速率限制
import random  # 导入随机库，用于在示例查询中随机选择带图片的文档

# --- 数据加载与图片关联函数 ---

def load_data_from_json_and_associate_images(json_path: str, image_dir: str) -> List[Dict]:
    """
    从指定的 JSON 文件加载文档的元数据 (如 ID 和描述文本)，
    并根据文档 ID (JSON中的 'name' 字段) 在指定的图像目录中查找并关联对应的图像文件。
    函数假设图像文件名是文档 ID 加上常见的图片扩展名 (如 .png, .jpg)。

    Args:
        json_path (str): 包含文档元数据的 JSON 文件路径。
                         JSON 文件应为一个列表，其中每个对象至少包含 'name' 和 'description' 字段。
        image_dir (str): 存放与 JSON 数据对应的图片文件的目录路径。

    Returns:
        List[Dict]: 一个包含处理后文档信息的字典列表。
                    每个字典包含以下键：
                    - 'id': 文档的唯一标识符 (来自 JSON 'name' 字段，确保为字符串)。
                    - 'text': 文档的文本描述 (来自 JSON 'description' 字段，确保为字符串或 None)。
                    - 'image_path': 找到的对应图像文件的完整路径 (str)。如果未找到图像或 image_dir 无效，则为 None。
                    返回空列表 ([]) 如果 JSON 文件不存在、无法解析或读取失败。
    """
    print(f"[数据加载] 正在从 '{json_path}' 加载数据并在 '{image_dir}' 中关联图像...")
    # 检查 JSON 文件是否存在
    if not os.path.exists(json_path):
        print(f"[数据加载] 错误: 未找到 JSON 文件 '{json_path}'。请检查文件路径。")
        return [] # 文件不存在，无法继续，返回空列表

    documents = [] # 用于存储处理后的文档信息
    # 定义常见的图像文件扩展名，用于查找图像文件
    image_extensions = ['.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff']

    # 尝试打开并解析 JSON 文件
    try:
        with open(json_path, 'r', encoding='utf-8') as f: # 使用 utf-8 编码打开
            json_data = json.load(f) # 解析 JSON 数据
    except json.JSONDecodeError as e:
        print(f"[数据加载] 错误: JSON 文件 '{json_path}' 解析失败: {e}")
        print(f"[数据加载] 请确保文件内容是有效的 JSON 格式。")
        return [] # JSON 格式错误，返回空列表
    except Exception as e:
        print(f"[数据加载] 错误: 读取 JSON 文件 '{json_path}' 失败: {e}")
        return [] # 其他读取错误，返回空列表

    print(f"[数据加载] 已成功从 '{json_path}' 加载 {len(json_data)} 条原始记录。")
    found_images_count = 0 # 计数器：成功关联到图像的文档数量
    missing_key_count = 0 # 计数器：因缺少 'name' 或 'description' 而跳过的记录数量

    # 遍历从 JSON 文件加载的每条记录
    for item in json_data:
        doc_id = item.get('name') # 获取 'name' 字段作为文档 ID
        text_content = item.get('description') # 获取 'description' 字段作为文本内容

        # 检查关键字段是否存在，不存在则跳过该记录
        if not doc_id or not text_content:
            missing_key_count += 1
            # 打印警告信息，帮助调试数据问题 (可选，如果记录很多可能刷屏)
            # print(f"[数据加载] 警告: 跳过一条记录，缺少 'name' 或 'description': {item}")
            continue # 继续处理下一条记录

        image_path = None # 初始化图像路径为 None
        # 检查图像目录是否存在且有效
        if image_dir and os.path.exists(image_dir):
            # 尝试不同的图像扩展名来查找匹配的图像文件
            for ext in image_extensions:
                # 构建潜在的图像文件名 (文档ID + 扩展名)
                potential_image_filename = str(doc_id) + ext
                # 构建完整的图像文件路径
                potential_image_path = os.path.join(image_dir, potential_image_filename)
                # 检查该路径对应的文件是否存在
                if os.path.exists(potential_image_path):
                    image_path = potential_image_path # 找到图像，记录其路径
                    found_images_count += 1 # 增加找到图像的计数
                    break # 找到一个匹配后就不再检查其他扩展名

        # 将处理后的文档信息添加到 documents 列表中
        documents.append({
            'id': str(doc_id), # 确保文档 ID 是字符串类型
            'text': str(text_content) if text_content is not None else None, # 确保文本是字符串，如果原始为 None 则保持 None
            'image_path': image_path # 存储找到的图像路径或 None
        })

    # 打印加载总结信息
    print(f"[数据加载] 成功准备了 {len(documents)} 个文档用于后续处理。")
    if missing_key_count > 0:
        print(f"[数据加载] 警告: 在原始 JSON 数据中，有 {missing_key_count} 条记录因缺少 'name' 或 'description' 字段而被跳过。")
    print(f"[数据加载] 其中 {found_images_count} 个文档成功关联了图像文件。")
    if len(documents) > 0 and found_images_count == 0 and image_dir:
         print(f"[数据加载] 提示: 未找到任何图像文件。请检查 '{image_dir}' 目录内容以及 JSON 'name' 字段与图像文件名的匹配规则 (应为 'name.extension')。")
    print(f"[数据加载] --- 数据加载与关联完毕 ---")
    return documents


# --- 1. 多模态编码器 (MultimodalEncoder) - 负责将文本和图像转换为向量表示 ---

class MultimodalEncoder:
    """
    使用 Hugging Face Transformers 库中的 CLIP 模型来对文本和/或图像进行编码。
    CLIP 模型能够将文本和图像映射到同一个高维向量空间，使得它们的向量表示具有语义上的可比性。
    这个类封装了模型的加载、输入预处理、编码过程以及向量归一化。

    核心功能:
    - 加载预训练的 CLIP 模型和对应的处理器。
    - 提供 `encode` 方法，可以接受文本、图像路径，或两者都接受。
    - `encode` 方法返回一个字典，包含各自的归一化向量 (text_vector, image_vector) 以及两者的归一化平均向量 (mean_vector)。
    - 自动检测并使用 GPU (如果可用)，否则使用 CPU。
    - 对生成的向量进行 L2 归一化，这对于后续使用 Faiss 进行基于内积 (余弦相似度) 的搜索非常重要。
    """
    def __init__(self, model_name: str = "openai/clip-vit-base-patch32"):
        """
        初始化 MultimodalEncoder 类。

        Args:
            model_name (str): 指定要加载的 Hugging Face Hub 上的 CLIP 模型名称。
                              例如 "openai/clip-vit-base-patch32"。
                              不同的模型有不同的性能和向量维度。
        Raises:
            Exception: 如果模型加载失败 (例如，网络问题、模型名称错误、未安装依赖)，会抛出异常。
                       模型加载是关键步骤，失败则编码器无法工作。
        """
        print(f"[Encoder] 初始化 MultimodalEncoder，尝试加载 CLIP 模型: {model_name}")
        try:
            # 加载与指定模型关联的处理器 (Processor)
            # 处理器负责将原始文本和图像转换为模型所需的输入格式 (tokenization, resizing, normalization等)
            self.processor = CLIPProcessor.from_pretrained(model_name)
            # 加载预训练的 CLIP 模型本身
            self.model = CLIPModel.from_pretrained(model_name)

            # 获取模型的向量维度 (对于 CLIP，文本和图像的输出维度通常相同)
            # 这将用于后续创建 Faiss 索引
            self.vector_dimension = self.model.text_model.config.hidden_size
            print(f"[Encoder] CLIP 模型和处理器加载成功。向量维度: {self.vector_dimension}")

            # 将模型设置为评估模式 (.eval())
            # 这会禁用 dropout 和 batch normalization 的更新，确保推理结果的一致性
            self.model.eval()

            # 检测可用的计算设备 (GPU 或 CPU)
            if torch.cuda.is_available():
                self.device = torch.device("cuda") # 如果有可用的 CUDA GPU，则使用 GPU
                print("[Encoder] 检测到 CUDA 支持，模型将运行在 GPU 上以加速计算。")
            else:
                self.device = torch.device("cpu")  # 否则使用 CPU
                print("[Encoder] 未检测到 CUDA 支持，模型将运行在 CPU 上 (可能较慢)。")
            # 将模型参数和缓冲区移动到选定的设备
            self.model.to(self.device)
            print("[Encoder] MultimodalEncoder 初始化完成。")

        except Exception as e:
             print(f"[Encoder] 错误: 加载 CLIP 模型 '{model_name}' 失败: {e}")
             print("[Encoder] 请确认：")
             print("  1. 模型名称正确且在 Hugging Face Hub 上可用。")
             print("  2. 已正确安装 'transformers', 'torch', 'pillow' 库 (pip install transformers torch pillow)。")
             print("  3. 网络连接正常，可以从 Hugging Face 下载模型文件。")
             raise e # 重新抛出异常，因为编码器无法在没有模型的情况下工作

    def _normalize_vector(self, vector: np.ndarray) -> np.ndarray:
        """
        对输入的 NumPy 向量进行 L2 范数归一化。
        归一化后的向量长度为 1 (在 L2 范数下)，这对于计算余弦相似度非常重要。
        使用内积 (Inner Product) 计算两个归一化向量的相似度等价于计算它们的余弦相似度。

        Args:
            vector (np.ndarray): 需要归一化的 Numpy 浮点数向量。

        Returns:
            np.ndarray: 经过 L2 归一化后的向量。如果输入向量接近零向量，则返回零向量以避免除以零。
        """
        # 计算向量的 L2 范数 (向量的欧几里得长度)
        norm = np.linalg.norm(vector)
        # 检查范数是否大于一个很小的阈值，以避免除以零或接近零的数导致数值不稳定
        if norm > 1e-9: # 1e-9 是一个小的正数阈值
            # 向量除以其范数得到归一化向量
            return vector / norm
        else:
            # 如果范数非常小 (向量接近零向量)，直接返回一个相同形状的零向量
            # print("[Encoder] 警告: 尝试归一化一个接近零的向量。返回零向量。") # 此警告可能过于频繁，注释掉
            return np.zeros_like(vector)

    def encode(self, text: Optional[str] = None, image_path: Optional[str] = None) -> Dict[str, Optional[np.ndarray]]:
        """
        对输入的文本和/或图像文件路径进行编码，生成对应的向量表示。

        Args:
            text (Optional[str]): 需要编码的文本字符串。如果为 None，则不进行文本编码。
            image_path (Optional[str]): 需要编码的图像文件的路径。如果为 None，则不进行图像编码。

        Returns:
            Dict[str, Optional[np.ndarray]]: 一个字典，包含以下可能的键值对：
                - 'text_vector': 如果提供了文本且编码成功，则为文本的归一化 NumPy 向量 (float32)。否则为 None。
                - 'image_vector': 如果提供了图像路径、图像文件有效且编码成功，则为图像的归一化 NumPy 向量 (float32)。否则为 None。
                - 'mean_vector': 如果文本和图像都提供了，并且两者都成功编码，则为两者向量的归一化平均向量 (float32)。否则为 None。
                                 这个平均向量可以作为文本和图像结合的多模态表示。
            如果 text 和 image_path 都为 None，将打印错误并返回所有值为 None 的字典。
        """
        # 输入检查：必须至少提供文本或图像路径之一
        if text is None and image_path is None:
            print("[Encoder] 错误: 必须提供文本或图像路径才能进行编码。")
            # 返回包含所有 None 值的字典，让调用者知道编码未进行
            return {'text_vector': None, 'image_vector': None, 'mean_vector': None}

        text_vector = None   # 初始化文本向量为 None
        image_vector = None  # 初始化图像向量为 None
        mean_vector = None   # 初始化平均向量为 None
        encoded_vectors = [] # 用于收集成功编码的向量，以便计算平均值

        # 使用 torch.no_grad() 上下文管理器进行推理
        # 这可以禁用梯度计算，减少内存消耗并加速计算，因为在编码（推理）阶段不需要反向传播
        with torch.no_grad():
            # --- 编码文本 (如果提供了文本) ---
            if text is not None:
                try:
                    # 1. 预处理文本: 使用 CLIP Processor 将文本转换为模型所需的输入格式 (token IDs, attention mask等)
                    #    `return_tensors="pt"` 表示返回 PyTorch 张量
                    #    `padding=True` 会将文本填充到批次中的最大长度
                    #    `truncation=True` 会将超过模型最大长度的文本截断
                    #    `.to(self.device)` 将输入张量移动到之前确定的计算设备 (CPU 或 GPU)
                    text_inputs = self.processor(text=text, return_tensors="pt", padding=True, truncation=True).to(self.device)

                    # 2. 获取文本特征: 调用模型的 `get_text_features` 方法获取文本的向量表示
                    text_features = self.model.get_text_features(**text_inputs) # 使用 ** 解包字典作为关键字参数

                    # 3. 后处理:
                    #    `.squeeze()` 移除批次维度 (如果批次大小为 1)
                    #    `.cpu()` 将结果张量移回 CPU (因为 NumPy 在 CPU 上工作)
                    #    `.numpy()` 将 PyTorch 张量转换为 NumPy 数组
                    #    `.astype('float32')` 确保数据类型为 float32，这是 Faiss 常用的类型
                    text_vector_raw = text_features.squeeze().cpu().numpy().astype('float32')

                    # 4. 归一化: 对原始向量进行 L2 归一化
                    text_vector = self._normalize_vector(text_vector_raw)

                    # 5. 收集: 将归一化后的文本向量添加到列表中，用于后续可能的平均计算
                    encoded_vectors.append(text_vector)
                    # print("[Encoder] 文本编码成功。") # 日志过于频繁，注释掉

                except Exception as e:
                    # 如果文本编码过程中出现任何错误，打印错误信息
                    print(f"[Encoder] 编码文本时发生错误: {e}")
                    # 保持 text_vector 为 None，表示编码失败

            # --- 编码图像 (如果提供了图像路径) ---
            if image_path is not None:
                try:
                    # 1. 加载图像: 使用 Pillow (PIL) 打开图像文件
                    #    `.convert("RGB")` 确保图像是 RGB 格式，这是 CLIP 模型通常需要的格式
                    image = Image.open(image_path).convert("RGB")

                    # 2. 预处理图像: 使用 CLIP Processor 将图像转换为模型所需的输入格式 (调整大小, 归一化等)
                    #    `return_tensors="pt"` 返回 PyTorch 张量
                    #    `.to(self.device)` 将输入张量移动到计算设备
                    image_inputs = self.processor(images=image, return_tensors="pt").to(self.device)

                    # 3. 获取图像特征: 调用模型的 `get_image_features` 方法获取图像的向量表示
                    image_features = self.model.get_image_features(**image_inputs)

                    # 4. 后处理: 与文本类似，转换为 NumPy float32 数组
                    image_vector_raw = image_features.squeeze().cpu().numpy().astype('float32')

                    # 5. 归一化: 对原始向量进行 L2 归一化
                    image_vector = self._normalize_vector(image_vector_raw)

                    # 6. 收集: 将归一化后的图像向量添加到列表中
                    encoded_vectors.append(image_vector)
                    # print(f"[Encoder] 图像 '{os.path.basename(image_path)}' 编码成功。") # 日志过于频繁，注释掉

                except FileNotFoundError:
                    # 如果图像文件不存在，打印警告信息
                    print(f"[Encoder] 警告: 图像文件未找到: {image_path}。将跳过此图像的编码。")
                    # 保持 image_vector 为 None
                except Exception as e:
                    # 如果处理图像时发生其他错误 (例如，图像文件损坏)，打印错误信息
                    print(f"[Encoder] 处理图像 {image_path} 时发生错误: {e}。将跳过此图像的编码。")
                    # 保持 image_vector 为 None

        # --- 计算平均向量 (仅当文本和图像都成功编码时) ---
        # 检查 text_vector 和 image_vector 是否都成功生成 (即不为 None)
        if text_vector is not None and image_vector is not None:
            try:
                # 1. 计算平均: 使用 NumPy 的 `mean` 函数计算两个向量的逐元素平均值
                #    `axis=0` 表示沿着第一个轴（向量本身）计算平均值
                #    确保结果是 float32 类型
                mean_vector_raw = np.mean([text_vector, image_vector], axis=0).astype('float32')

                # 2. 归一化: 对计算出的平均向量再次进行 L2 归一化
                #    这很重要，因为平均操作本身不保证结果向量的长度为 1
                mean_vector = self._normalize_vector(mean_vector_raw)
                # print("[Encoder] 平均向量计算成功。") # 日志过于频繁，注释掉
            except Exception as e:
                # 如果计算平均向量时出错，打印错误信息
                print(f"[Encoder] 计算平均向量时发生错误: {e}")
                # 保持 mean_vector 为 None

        # 返回包含所有结果向量的字典
        return {
            'text_vector': text_vector,
            'image_vector': image_vector,
            'mean_vector': mean_vector
        }


# --- 2. 索引器 (Indexer) - 负责文档处理、向量化、构建索引和存储数据 ---

class Indexer:
    """
    Indexer 类是 RAG 系统的数据核心。它负责：
    1.  **接收文档数据**: 从 `load_data_from_json_and_associate_images` 获取文档列表。
    2.  **调用编码器**: 使用 `MultimodalEncoder` 对每个文档的文本和/或图像进行向量化。
    3.  **存储元数据**: 将文档的原始信息（ID、文本、图像路径）存储在 SQLite 数据库中。
        数据库还维护一个自增的 `internal_id`，用作 Faiss 索引的唯一标识符。
    4.  **构建和管理向量索引**:
        -   创建并维护 **三个独立** 的 Faiss 索引：一个用于文本向量，一个用于图像向量，一个用于平均向量。
        -   使用 `Faiss IndexIDMap2`，允许将向量与我们自定义的 `internal_id` (来自数据库) 关联起来。
        -   索引使用内积 (`IndexFlatIP`) 作为相似度度量，适用于已归一化的向量 (等价于余弦相似度)。
    5.  **持久化**: 能够加载已存在的索引文件和数据库，或在首次运行时创建它们，并在结束时保存索引。

    这种分离索引的设计允许根据查询类型（纯文本、纯图像、多模态）选择最合适的索引进行搜索。
    """
    def __init__(self,
                 db_path: str = 'multimodal_rag_data.db', # SQLite 数据库文件路径
                 faiss_text_index_path: str = 'text_index.faiss', # 存储文本向量的 Faiss 索引文件路径
                 faiss_image_index_path: str = 'image_index.faiss', # 存储图像向量的 Faiss 索引文件路径
                 faiss_mean_index_path: str = 'mean_index.faiss', # 存储平均向量的 Faiss 索引文件路径
                 clip_model_name: str = "openai/clip-vit-base-patch32"): # 使用的 CLIP 模型名称
        """
        初始化 Indexer 实例。

        Args:
            db_path (str): 指定 SQLite 数据库文件的保存路径。
            faiss_text_index_path (str): 指定文本向量 Faiss 索引文件的保存路径。
            faiss_image_index_path (str): 指定图像向量 Faiss 索引文件的保存路径。
            faiss_mean_index_path (str): 指定平均向量 Faiss 索引文件的保存路径。
            clip_model_name (str): 传递给 `MultimodalEncoder` 的 CLIP 模型名称。
                                   必须与用于查询编码的模型保持一致。
        """
        print("[Indexer] 初始化 Indexer...")
        self.db_path = db_path
        self.faiss_text_index_path = faiss_text_index_path
        self.faiss_image_index_path = faiss_image_index_path
        self.faiss_mean_index_path = faiss_mean_index_path

        # 步骤 1: 初始化多模态编码器
        print("[Indexer]   - 正在初始化 MultimodalEncoder...")
        # Indexer 内部拥有一个 Encoder 实例，用于对其接收的文档进行编码
        self.encoder = MultimodalEncoder(clip_model_name)
        # 获取编码器产生的向量维度，用于创建 Faiss 索引
        self.vector_dimension = self.encoder.vector_dimension
        print(f"[Indexer]   - Encoder 初始化完成 (向量维度: {self.vector_dimension})。")

        # 步骤 2: 初始化 SQLite 数据库 (用于存储元数据)
        print("[Indexer]   - 正在初始化 SQLite 数据库...")
        # 调用私有方法 _init_db 来创建表结构（如果尚不存在）
        self._init_db()
        print("[Indexer]   - 数据库初始化完成。")

        # 步骤 3: 加载或创建三个 Faiss 向量索引
        print("[Indexer]   - 正在加载或创建 Faiss 向量索引...")
        # 分别为文本、图像和平均向量加载或创建 Faiss 索引
        # _load_or_create_faiss_index 方法会处理文件存在性检查和新索引创建
        self.text_index = self._load_or_create_faiss_index(self.faiss_text_index_path, "文本")
        self.image_index = self._load_or_create_faiss_index(self.faiss_image_index_path, "图像")
        self.mean_index = self._load_or_create_faiss_index(self.faiss_mean_index_path, "平均")
        print("[Indexer]   - Faiss 索引准备就绪。")

        print("[Indexer] Indexer 初始化完成。")


    def _init_db(self):
        """
        初始化 SQLite 数据库连接并创建所需的 'documents' 表（如果它还不存在）。
        这个表用于存储文档的元数据，并将原始文档 ID (doc_id) 映射到数据库生成的
        自增主键 `internal_id`，这个 `internal_id` 将作为 Faiss 索引中向量的 ID。
        """
        print(f"[Indexer] 正在连接并初始化数据库表结构于: '{self.db_path}'...")
        try:
            # 使用 'with' 语句确保数据库连接在使用后自动关闭
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor() # 获取数据库游标，用于执行 SQL 命令
                # 创建 'documents' 表的 SQL 语句
                # IF NOT EXISTS 确保如果表已存在，则不会尝试重新创建，避免错误
                cursor.execute('''
                    CREATE TABLE IF NOT EXISTS documents (
                        internal_id INTEGER PRIMARY KEY AUTOINCREMENT, -- 数据库自动生成的整数主键，也将用作 Faiss 索引中的向量 ID
                        doc_id TEXT UNIQUE NOT NULL,                 -- 原始文档的唯一标识符 (例如来自 JSON 的 'name')，必须唯一且不能为空
                        text TEXT,                                   -- 文档的文本内容，允许为空
                        image_path TEXT                              -- 关联的图像文件路径，允许为空
                    )
                ''')
                # 可选：在 doc_id 列上创建索引，可以加快通过原始 doc_id 查找记录的速度
                # 这在 index_documents 中检查重复项时很有用
                cursor.execute("CREATE INDEX IF NOT EXISTS idx_doc_id ON documents (doc_id)")
                conn.commit() # 提交事务，使表结构更改生效
                print(f"[Indexer] 数据库表 'documents' 初始化成功 (或已存在)。")
        except Exception as e:
             print(f"[Indexer] 错误: 初始化数据库 '{self.db_path}' 失败: {e}")
             raise e # 数据库是核心依赖，初始化失败则无法继续，抛出异常

    def _load_or_create_faiss_index(self, index_path: str, index_type: str) -> faiss.Index:
        """
        尝试从指定路径加载一个 Faiss 索引文件。
        如果文件存在且向量维度匹配，则加载它。
        如果文件不存在，或者文件存在但维度不匹配当前编码器，则创建一个新的空索引。
        使用 `IndexIDMap2` 来存储带有自定义 64 位整数 ID 的向量。

        Args:
            index_path (str): Faiss 索引文件的路径。
            index_type (str): 索引类型的描述性名称 (例如 "文本", "图像", "平均")，用于日志记录。

        Returns:
            faiss.Index: 加载的或新创建的 Faiss 索引对象 (具体类型为 IndexIDMap2)。
        """
        try:
            # 检查索引文件是否已存在
            if os.path.exists(index_path):
                print(f"[Indexer] 发现现有的 {index_type} Faiss 索引文件，尝试加载: {index_path}")
                # 使用 faiss.read_index 读取索引文件
                index = faiss.read_index(index_path)
                # **重要**: 检查加载的索引维度 (`index.d`) 是否与当前编码器模型产生的向量维度 (`self.vector_dimension`) 一致
                if index.d != self.vector_dimension:
                    # 如果维度不匹配，说明索引是使用不同模型创建的，不能直接使用
                    print(f"[Indexer] 警告: 加载的 {index_type} 索引维度 ({index.d}) 与当前模型配置的维度 ({self.vector_dimension}) 不匹配！")
                    print(f"[Indexer] 这可能意味着之前的索引是用不同的模型创建的。将忽略旧索引并创建一个新的空 {index_type} 索引。")
                    # 创建一个新的空索引替换掉加载的旧索引
                    index = self._create_new_faiss_index(index_type)
                else:
                    # 维度匹配，加载成功
                    print(f"[Indexer] 成功加载 {index_type} Faiss 索引，包含 {index.ntotal} 个向量。")
            else:
                # 索引文件不存在
                print(f"[Indexer] 未找到 {index_type} Faiss 索引文件: {index_path}。将创建一个新的空索引。")
                # 调用内部方法创建新的空索引
                index = self._create_new_faiss_index(index_type)
        except Exception as e:
            # 处理加载或读取索引文件时可能发生的任何错误
            print(f"[Indexer] 错误: 加载或处理 {index_type} Faiss 索引 '{index_path}' 失败: {e}")
            print(f"[Indexer] 将创建一个新的空 {index_type} 索引作为安全回退。")
            # 即使加载失败，也创建一个新的空索引，保证程序能继续（虽然没有历史数据）
            index = self._create_new_faiss_index(index_type)
        return index

    def _create_new_faiss_index(self, index_type: str) -> faiss.Index:
         """
         创建一个新的、空的 Faiss 索引。
         配置为使用内积 (`IndexFlatIP`) 进行搜索，并使用 `IndexIDMap2` 包装器来支持自定义 64 位整数 ID。
         `IndexFlatIP` 适用于存储原始向量并进行精确的暴力内积搜索。对于归一化的向量，内积等价于余弦相似度。

         Args:
             index_type (str): 索引类型的描述 (用于日志)。

         Returns:
             faiss.Index: 新创建的、空的 Faiss IndexIDMap2 索引对象。
         """
         # 1. 创建基础索引 (quantizer): IndexFlatIP
         #    - IndexFlat: 表示存储完整的、未经压缩的向量。
         #    - IP: 表示使用内积 (Inner Product) 作为距离/相似度度量。
         #    适用于向量已经 L2 归一化的情况，此时内积得分等于余弦相似度。
         quantizer = faiss.IndexFlatIP(self.vector_dimension)

         # 2. 创建 ID 映射包装器: IndexIDMap2
         #    - 它包装了一个基础索引 (quantizer)，并允许我们将每个添加的向量与一个我们自己指定的 64 位整数 ID 相关联。
         #    - '2' 表示它将内部 ID 重新映射，通常更灵活。
         #    - 我们将使用数据库生成的 `internal_id` 作为这个自定义 ID。
         index = faiss.IndexIDMap2(quantizer)

         print(f"[Indexer] 已成功创建一个新的、空的 {index_type} Faiss 索引 (类型: IndexIDMap2 包裹 IndexFlatIP)。")
         print(f"          索引维度: {self.vector_dimension}, 相似度度量: 内积 (适用于归一化向量的余弦相似度)。")
         return index

    def index_documents(self, documents: List[Dict]):
        """
        核心索引流程：处理一个文档列表，对每个文档进行编码，将元数据存入数据库，
        并将生成的向量（文本、图像、平均）添加到对应的 Faiss 索引中。
        此方法会处理重复文档（基于 `doc_id`），并进行批量 Faiss 添加以提高效率。

        Args:
            documents (List[Dict]): 一个字典列表，其中每个字典代表一个待索引的文档，
                                    应包含 'id' (原始文档ID), 'text', 'image_path' 键。
                                    这是 `load_data_from_json_and_associate_images` 的输出格式。
        """
        # 检查输入列表是否为空
        if not documents:
            print("[Indexer] 未提供任何文档进行索引操作。")
            return

        print(f"[Indexer] 开始索引流程，准备处理 {len(documents)} 个文档...")
        # 初始化列表，用于收集需要批量添加到 Faiss 的向量和对应的 ID
        # 分别为文本、图像和平均向量准备
        text_vectors_batch = []   # 存储文本向量 (numpy arrays)
        text_ids_batch = []       # 存储对应的 internal_id (integers)
        image_vectors_batch = []  # 存储图像向量
        image_ids_batch = []      # 存储对应的 internal_id
        mean_vectors_batch = []   # 存储平均向量
        mean_ids_batch = []       # 存储对应的 internal_id

        # 初始化计数器，用于跟踪索引过程的状态
        processed_count = 0          # 成功处理并尝试编码的文档数
        skipped_duplicate_count = 0  # 因 doc_id 已存在而跳过的文档数
        encoding_error_count = 0     # 因编码阶段出错而未能添加向量的文档数
        db_error_count = 0           # 因数据库操作出错而跳过的文档数

        conn = None # 确保 conn 在 try 块外部可见，以便在 finally 中关闭

        try:
            # --- 数据库连接和事务管理 ---
            # 建立数据库连接
            conn = sqlite3.connect(self.db_path)
            # 获取数据库游标
            cursor = conn.cursor()
            # 手动管理事务：默认情况下，sqlite3 的 execute 会隐式开始事务。
            # 我们希望整个批次的插入和 Faiss 添加要么全部成功，要么全部回滚（如果中途出错）。
            # 虽然 Faiss 操作本身不是事务性的，但至少保证数据库的元数据是一致的。
            # 在循环外开始事务不是标准做法，但可以通过在末尾 commit/rollback 实现类似效果。
            # 更安全的做法是在循环内每次插入后不 commit，最后统一 commit。

            # --- 遍历每个待索引的文档 ---
            for i, doc in enumerate(documents):
                doc_id = doc.get('id')          # 获取原始文档 ID
                text = doc.get('text')          # 获取文本内容
                image_path = doc.get('image_path') # 获取图像路径

                # 基本验证：必须有 doc_id，且至少有文本或图像路径之一
                if not doc_id:
                    print(f"[Indexer] 警告: 跳过第 {i+1} 条记录，因缺少 'id' 字段: {doc}")
                    continue
                if not text and not image_path:
                     print(f"[Indexer] 警告: 跳过文档 '{doc_id}'，因为它既没有文本内容也没有关联的图像路径。")
                     continue

                # --- 1. 检查文档是否已存在 (基于 doc_id) ---
                # 查询数据库中是否已存在具有相同 doc_id 的记录
                cursor.execute("SELECT internal_id FROM documents WHERE doc_id = ?", (doc_id,))
                existing_record = cursor.fetchone() # 获取查询结果（如果存在）
                if existing_record:
                     # 如果记录已存在，打印提示（可注释掉以减少输出），增加跳过计数，然后处理下一个文档
                     # print(f"[Indexer] 文档 ID '{doc_id}' 已存在于数据库中 (internal_id: {existing_record[0]})。跳过此重复文档。")
                     skipped_duplicate_count += 1
                     continue # 跳到下一个文档

                # --- 2. 插入元数据到数据库，获取 internal_id ---
                internal_id = None # 初始化 internal_id
                try:
                    # 执行 INSERT 语句将新文档的元数据插入到 'documents' 表
                    cursor.execute(
                        "INSERT INTO documents (doc_id, text, image_path) VALUES (?, ?, ?)",
                        (doc_id, text, image_path) # 使用参数化查询防止 SQL 注入
                    )
                    # 获取刚刚插入行的自增主键 (internal_id)
                    internal_id = cursor.lastrowid
                    # 在某些罕见情况下，lastrowid 可能不可用，增加一个后备查询来确保获取 ID
                    if internal_id is None:
                        print(f"[Indexer] 警告: cursor.lastrowid 未返回有效的 internal_id for doc_id '{doc_id}'. 尝试重新查询...")
                        cursor.execute("SELECT internal_id FROM documents WHERE doc_id = ?", (doc_id,))
                        result = cursor.fetchone()
                        if result:
                            internal_id = result[0]
                        else:
                           # 如果还是获取不到，这是一个严重问题
                           raise sqlite3.Error(f"严重错误：无法获取新插入文档 '{doc_id}' 的 internal_id！数据库可能处于不一致状态。")
                    # print(f"[Indexer] 文档 '{doc_id}' 元数据已插入数据库，获得 internal_id: {internal_id}") # 过于频繁，注释掉

                except sqlite3.IntegrityError:
                    # 理论上不应发生，因为前面已经检查过 doc_id 的唯一性。
                    # 但如果存在并发写入或其他逻辑错误，可能会触发 UNIQUE 约束。
                    print(f"[Indexer] 数据库完整性错误: 尝试插入已存在的 doc_id '{doc_id}' (可能是并发问题或逻辑错误?)。跳过此文档。")
                    skipped_duplicate_count += 1
                    conn.rollback() # 回滚当前失败的插入尝试
                    continue # 跳到下一个文档
                except sqlite3.Error as db_e:
                    # 处理其他可能的数据库错误 (例如磁盘满、权限问题等)
                    print(f"[Indexer] 数据库错误 (文档 '{doc_id}'): {db_e}。将跳过此文档的索引。")
                    db_error_count += 1
                    conn.rollback() # 回滚
                    continue # 跳到下一个文档

                # --- 3. 使用 Encoder 进行多模态向量化 ---
                encoded_data = None # 初始化编码结果
                try:
                    # 调用 encoder 的 encode 方法，传入文本和图像路径
                    # print(f"[Indexer] 正在编码文档 {doc_id} (internal_id: {internal_id})...") # 过于频繁，注释掉
                    encoded_data = self.encoder.encode(text=text, image_path=image_path)
                    # `encoded_data` 是一个字典: {'text_vector': vec|None, 'image_vector': vec|None, 'mean_vector': vec|None}
                except Exception as encode_e:
                    # 处理编码过程中可能发生的任何异常
                    print(f"[Indexer] 错误: 编码文档 '{doc_id}' (internal_id: {internal_id}) 时发生错误: {encode_e}。")
                    print(f"[Indexer] 注意: 此文档的元数据已存入数据库，但其向量将不会被添加到 Faiss 索引中。")
                    encoding_error_count += 1
                    # 不需要回滚数据库，因为元数据存储本身是成功的。只是缺少对应的向量。
                    # 检索时，如果通过 ID 查找到这个文档，但 Faiss 中没有对应向量，它就不会被相似度搜索找到。
                    continue # 跳过后续的向量添加步骤，处理下一个文档

                # --- 4. 将成功编码的向量添加到批处理列表 ---
                # 检查编码结果字典是否存在，以及各个向量是否成功生成 (不为 None)
                if encoded_data:
                    if encoded_data['text_vector'] is not None:
                        # 如果文本向量存在，将其和对应的 internal_id 加入文本批处理列表
                        text_vectors_batch.append(encoded_data['text_vector'])
                        text_ids_batch.append(internal_id)
                    if encoded_data['image_vector'] is not None:
                        # 如果图像向量存在，将其和对应的 internal_id 加入图像批处理列表
                        image_vectors_batch.append(encoded_data['image_vector'])
                        image_ids_batch.append(internal_id)
                    if encoded_data['mean_vector'] is not None:
                        # 如果平均向量存在，将其和对应的 internal_id 加入平均向量批处理列表
                        mean_vectors_batch.append(encoded_data['mean_vector'])
                        mean_ids_batch.append(internal_id)

                    # 只有在至少一个向量被成功编码并准备添加时，才增加 processed_count
                    if encoded_data['text_vector'] is not None or \
                       encoded_data['image_vector'] is not None or \
                       encoded_data['mean_vector'] is not None:
                        processed_count += 1

                # 可以在这里添加一个小的延时，如果需要观察过程或避免某些资源竞争 (通常不需要)
                # time.sleep(0.01)

            # --- 文档遍历和初步处理完成 ---
            print(f"[Indexer] 所有文档已遍历处理。准备将收集到的向量批量添加到 Faiss 索引中...")
            print(f"          待添加: {len(text_ids_batch)} 个文本向量, {len(image_ids_batch)} 个图像向量, {len(mean_ids_batch)} 个平均向量。")

            # --- 5. 批量将向量和 ID 添加到对应的 Faiss 索引 ---
            # 使用 add_with_ids 方法可以一次性添加多个向量及其对应的 ID，比逐个添加高效得多。

            # 添加文本向量
            if text_vectors_batch: # 检查列表是否非空
                # Faiss 需要 ID 是 int64 类型的 NumPy 数组
                ids_np = np.array(text_ids_batch, dtype='int64')
                # Faiss 需要向量是 float32 类型的 NumPy 数组 (n_vectors, dimension)
                vectors_np = np.array(text_vectors_batch, dtype='float32')
                # 调用 Faiss 索引的 add_with_ids 方法
                self.text_index.add_with_ids(vectors_np, ids_np)
                print(f"[Indexer] 已成功向 Text Index 添加 {len(text_vectors_batch)} 个向量。当前总数: {self.text_index.ntotal}")

            # 添加图像向量
            if image_vectors_batch:
                ids_np = np.array(image_ids_batch, dtype='int64')
                vectors_np = np.array(image_vectors_batch, dtype='float32')
                self.image_index.add_with_ids(vectors_np, ids_np)
                print(f"[Indexer] 已成功向 Image Index 添加 {len(image_vectors_batch)} 个向量。当前总数: {self.image_index.ntotal}")

            # 添加平均向量
            if mean_vectors_batch:
                ids_np = np.array(mean_ids_batch, dtype='int64')
                vectors_np = np.array(mean_vectors_batch, dtype='float32')
                self.mean_index.add_with_ids(vectors_np, ids_np)
                print(f"[Indexer] 已成功向 Mean Index 添加 {len(mean_vectors_batch)} 个向量。当前总数: {self.mean_index.ntotal}")

            # --- 6. 提交数据库事务 ---
            # 如果所有操作（主要是数据库插入）都成功，并且没有中途因严重错误退出，
            # 则提交数据库事务，将所有 INSERT 操作持久化。
            conn.commit()
            print("[Indexer] 数据库事务已成功提交。")

        except Exception as e:
            # 捕获在整个索引流程中（循环内部或外部）可能发生的未预料的严重错误
            print(f"[Indexer] 严重错误: 在索引过程中发生意外异常: {e}")
            if conn:
                # 如果发生了严重错误，并且数据库连接仍然有效，则回滚事务
                # 这将撤销自上次 commit() 以来（在此代码中是整个批次开始以来）的所有数据库更改
                print("[Indexer] 检测到严重错误，正在回滚数据库更改...")
                try:
                    conn.rollback()
                    print("[Indexer] 数据库更改已成功回滚。")
                except Exception as rb_e:
                    print(f"[Indexer] 错误: 尝试回滚数据库事务失败: {rb_e}")
            # 抛出异常或记录更详细信息，可能需要根据具体情况决定是否让程序停止
            import traceback
            traceback.print_exc()
            # 注意：Faiss 的 add_with_ids 操作通常是原子性的，但 Faiss 本身不直接支持事务回滚。
            # 如果在 Faiss 添加过程中出错，部分向量可能已经添加。
            # 回滚数据库可以确保元数据与索引状态（可能部分添加）的不一致性主要体现在 Faiss 侧，
            # 而不是数据库中有指向不存在向量的记录。

        finally:
            # --- 清理资源 ---
            # 无论索引过程成功还是失败，最后都确保关闭数据库连接
            if conn:
                conn.close()
                # print("[Indexer] 数据库连接已关闭。") # 可选日志

        # --- 打印索引过程总结 ---
        print(f"\n[Indexer] --- 索引过程总结 ---")
        print(f"- 尝试处理的文档总数 (来自输入列表): {len(documents)}")
        print(f"- 成功处理并至少尝试编码的文档数: {processed_count}")
        print(f"- 因 'doc_id' 重复而跳过的文档数: {skipped_duplicate_count}")
        print(f"- 因编码阶段错误跳过向量添加的文档数: {encoding_error_count}")
        print(f"- 因数据库操作错误跳过的文档数: {db_error_count}")
        print(f"- 当前 Text Index 中的向量总数: {self.text_index.ntotal}")
        print(f"- 当前 Image Index 中的向量总数: {self.image_index.ntotal}")
        print(f"- 当前 Mean Index 中的向量总数: {self.mean_index.ntotal}")
        db_final_count = self.get_document_count() # 获取最终数据库中的记录数
        print(f"- 当前 SQLite 数据库中存储的文档元数据记录数: {db_final_count}")
        # 检查一致性（理论上，DB 记录数 >= Faiss 向量数之和中的最大值）
        if db_final_count < max(self.text_index.ntotal, self.image_index.ntotal, self.mean_index.ntotal):
             print(f"[Indexer] 警告: 数据库记录数 ({db_final_count}) 少于某个 Faiss 索引中的向量数。数据可能存在不一致！")
        print(f"[Indexer] --- 索引过程结束 ---")


    def get_document_by_internal_id(self, internal_id: int) -> Optional[Dict]:
        """
        根据 Faiss 搜索返回的 `internal_id` (数据库主键)，从 SQLite 数据库中检索对应的原始文档元数据。

        Args:
            internal_id (int): 要查询的文档在数据库中的 `internal_id`。

        Returns:
            Optional[Dict]: 包含文档信息的字典，键包括 'id' (原始 doc_id), 'text', 'image_path', 'internal_id'。
                            如果数据库中找不到对应 `internal_id` 的记录，则返回 None。
        """
        try:
            # 连接数据库
            with sqlite3.connect(self.db_path) as conn:
                # 设置 row_factory 为 sqlite3.Row
                # 这使得查询结果可以像字典一样通过列名访问，更方便
                conn.row_factory = sqlite3.Row
                cursor = conn.cursor()
                # 执行 SELECT 查询，根据 internal_id 查找记录
                cursor.execute(
                    "SELECT internal_id, doc_id, text, image_path FROM documents WHERE internal_id = ?",
                    (internal_id,) # 注意参数是元组形式
                )
                row = cursor.fetchone() # 获取一行结果
                # 如果找到了记录 (row 不为 None)
                if row:
                    # 将 sqlite3.Row 对象转换为标准的 Python 字典
                    doc_data = dict(row)
                    # 为了与之前的接口或数据结构保持一致，将 'doc_id' 重命名为 'id'
                    doc_data['id'] = doc_data.pop('doc_id')
                    return doc_data # 返回包含文档信息的字典
                else:
                    # 如果没有找到记录
                    return None # 返回 None
        except Exception as e:
             # 处理数据库查询过程中可能发生的错误
             print(f"[Indexer] 错误: 从数据库根据 internal_id {internal_id} 获取文档时出错: {e}")
             return None # 出错时返回 None

    def get_documents_by_internal_ids(self, internal_ids: List[int]) -> Dict[int, Dict]:
        """
        根据一个 `internal_id` 的列表，从 SQLite 数据库中批量检索对应的多个文档元数据。
        使用批量查询可以提高效率，尤其是在处理 Faiss 返回的 Top-K 结果时。

        Args:
            internal_ids (List[int]): 一个包含多个数据库 `internal_id` 的列表。

        Returns:
            Dict[int, Dict]: 一个字典，其中键是 `internal_id`，值是对应的文档数据字典
                             (包含 'id', 'text', 'image_path', 'internal_id')。
                             如果列表中的某个 ID 在数据库中找不到，则结果字典中不会包含该 ID 的条目。
                             如果输入列表为空，返回空字典。
        """
        # 如果输入的 ID 列表为空，直接返回空字典
        if not internal_ids:
            return {}

        results = {} # 初始化结果字典
        try:
            # 连接数据库
            with sqlite3.connect(self.db_path) as conn:
                conn.row_factory = sqlite3.Row # 使用 Row factory 使结果易于处理
                cursor = conn.cursor()

                # 构建 SQL 查询语句，使用 IN 操作符和参数占位符进行批量查询
                # 1. 创建占位符字符串: (?, ?, ..., ?) - 每个 ID 一个 '?'
                placeholders = ','.join('?' for _ in internal_ids)
                # 2. 构建完整的 SQL 查询语句
                query = f"SELECT internal_id, doc_id, text, image_path FROM documents WHERE internal_id IN ({placeholders})"

                # 执行查询，将 ID 列表作为参数传递
                cursor.execute(query, internal_ids)
                rows = cursor.fetchall() # 获取所有匹配的行

                # 遍历查询结果
                for row in rows:
                    # 将每行 (sqlite3.Row) 转换为字典
                    doc_data = dict(row)
                    # 重命名 'doc_id' 为 'id'
                    doc_data['id'] = doc_data.pop('doc_id')
                    # 使用 internal_id 作为键，将文档数据存入结果字典
                    results[doc_data['internal_id']] = doc_data
        except Exception as e:
             # 处理批量查询中的数据库错误
             print(f"[Indexer] 错误: 从数据库根据 internal_id 列表批量获取文档时出错: {e}")
             # 即使出错，也返回已经成功获取的部分结果 (results 字典)
        return results

    def get_document_count(self) -> int:
         """获取当前 SQLite 数据库 'documents' 表中存储的文档总数。"""
         try:
             # 连接数据库
             with sqlite3.connect(self.db_path) as conn:
                 cursor = conn.cursor()
                 # 执行 COUNT(*) 查询获取总行数
                 cursor.execute("SELECT COUNT(*) FROM documents")
                 # fetchone() 返回一个包含单个值的元组，例如 (50,)
                 count_result = cursor.fetchone()
                 # 提取元组中的计数值，如果查询无结果（理论上 COUNT(*) 总有结果，但做个健壮性检查）则返回 0
                 count = count_result[0] if count_result else 0
                 return count if count is not None else 0 # 再次确保返回的是整数 0
         except Exception as e:
              # 处理查询计数时的数据库错误
              print(f"[Indexer] 错误: 从数据库获取文档总数时出错: {e}")
              return 0 # 出错时返回 0

    def save_indices(self):
        """
        将内存中的所有三个 Faiss 索引（文本、图像、平均）分别保存到对应的文件路径中。
        这用于持久化索引，以便下次启动时可以加载，避免重新建立索引。
        只保存非空的索引。
        """
        print("[Indexer] 正在尝试保存所有 Faiss 索引到文件...")
        # 调用内部辅助方法分别保存每个索引
        self._save_single_index(self.text_index, self.faiss_text_index_path, "Text")
        self._save_single_index(self.image_index, self.faiss_image_index_path, "Image")
        self._save_single_index(self.mean_index, self.faiss_mean_index_path, "Mean")
        print("[Indexer] Faiss 索引保存操作完成。")

    def _save_single_index(self, index: faiss.Index, index_path: str, index_type: str):
        """
        辅助方法：保存单个 Faiss 索引到指定文件。
        仅当索引存在且包含至少一个向量时才执行保存。

        Args:
            index (faiss.Index): 要保存的 Faiss 索引对象。
            index_path (str): 保存索引的文件路径。
            index_type (str): 索引类型的描述 (用于日志)。
        """
        # 检查索引对象是否存在，是否有 'ntotal' 属性 (表示向量数量)，以及向量数量是否大于 0
        if hasattr(index, 'ntotal') and index.ntotal > 0:
            try:
                # 使用 faiss.write_index 将索引写入文件
                faiss.write_index(index, index_path)
                print(f"[Indexer]   - {index_type} Faiss 索引 (含 {index.ntotal} 向量) 已成功保存到: {index_path}")
            except Exception as e:
                # 处理保存索引时可能发生的错误 (例如，磁盘空间不足、权限问题)
                print(f"[Indexer]   - 错误: 保存 {index_type} Faiss 索引到 '{index_path}' 失败: {e}")
        elif hasattr(index, 'ntotal'):
             # 如果索引存在但为空 (ntotal == 0)，则跳过保存
             print(f"[Indexer]   - {index_type} Faiss 索引为空，跳过保存到 '{index_path}'。")
        else:
             # 如果索引对象无效或未正确初始化
             print(f"[Indexer]   - 警告: {index_type} Faiss 索引似乎未初始化或无效，无法保存。")

    def close(self):
        """
        关闭 Indexer 实例时调用的清理方法。
        主要职责是确保持久化所有 Faiss 索引。
        SQLite 连接是通过 `with` 语句管理的，在每个方法结束时会自动关闭，这里无需额外操作。
        Faiss 索引对象本身在 Python 中是内存对象，不需要显式关闭，保存即可。
        """
        print("[Indexer] 正在关闭 Indexer...")
        # 调用 save_indices 确保存储最新的索引状态
        self.save_indices()
        print("[Indexer] Indexer 关闭完成。")


# --- 3. 检索器 (Retriever) - 负责查询向量化、相似度搜索和上下文组织 ---

class Retriever:
    """
    Retriever 类负责处理用户的查询（可以是文本、图像或两者结合），
    利用与 Indexer 相同的 `MultimodalEncoder` 对查询进行向量化，
    然后根据查询的类型在 `Indexer` 提供的相应 Faiss 索引（文本、图像或平均）中执行相似性搜索，
    找出最相关的 Top-K 个文档的 `internal_id`。
    最后，它使用这些 `internal_id` 从 `Indexer` 的数据库中批量获取原始文档的元数据，
    并将这些包含原始信息和相似度得分的文档组织成一个列表返回。

    核心逻辑:
    - 接收查询 (str 或 dict)。
    - 使用 Encoder 对查询进行编码，得到查询向量 (文本、图像、平均)。
    - 根据查询类型选择最合适的 Faiss 索引 (Text, Image, Mean) 和对应的查询向量。
    - 在选定的 Faiss 索引中执行 Top-K 搜索，获取 `internal_id`s 和相似度得分。
    - 调用 Indexer 的 `get_documents_by_internal_ids` 批量获取文档元数据。
    - 将元数据与得分结合，返回排序后的结果列表。
    """
    def __init__(self, indexer: Indexer):
        """
        初始化 Retriever 实例。

        Args:
            indexer (Indexer): 一个已经初始化并包含了数据和索引的 Indexer 实例。
                               Retriever 严重依赖 Indexer 来访问编码器、Faiss 索引和数据库。

        Raises:
            ValueError: 如果传入的 `indexer` 不是有效的 Indexer 实例，或者该实例缺少必要的 Faiss 索引属性。
                        一个没有有效索引的 Retriever 是无法工作的。
        """
        print("[Retriever] 初始化 Retriever...")
        # 验证传入的 indexer 是否是 Indexer 类的实例
        if not isinstance(indexer, Indexer):
             raise ValueError("[Retriever] 错误: 初始化 Retriever 需要一个有效的 Indexer 实例。")
        # 验证 Indexer 实例是否已成功创建了所需的 Faiss 索引对象
        if not (hasattr(indexer, 'text_index') and hasattr(indexer, 'image_index') and hasattr(indexer, 'mean_index')):
            raise ValueError("[Retriever] 错误: 提供的 Indexer 实例似乎缺少一个或多个必需的 Faiss 索引对象 (text_index, image_index, mean_index)。请确保 Indexer 已成功初始化。")

        # 保存对传入的 Indexer 实例的引用
        self.indexer = indexer
        # 复用 Indexer 内部的 Encoder 实例，确保查询编码与文档编码使用相同的模型和设置
        self.encoder = self.indexer.encoder
        # 获取向量维度，用于后续 Faiss 搜索时验证或处理向量形状
        self.vector_dimension = self.indexer.vector_dimension

        # 获取对 Indexer 中三个 Faiss 索引的直接引用，方便访问
        self.text_index = self.indexer.text_index
        self.image_index = self.indexer.image_index
        self.mean_index = self.indexer.mean_index

        # 检查所有索引是否都为空，如果是，则检索无法返回结果
        total_vectors = self.text_index.ntotal + self.image_index.ntotal + self.mean_index.ntotal
        if total_vectors == 0:
             print("[Retriever] 警告: Indexer 中的所有 Faiss 索引当前都为空。检索操作将无法找到任何匹配项。")
        else:
             # 打印各索引的状态信息
             print(f"[Retriever] Retriever 初始化成功。关联的 Indexer 状态:")
             print(f"          - Text Index: {self.text_index.ntotal} 个向量")
             print(f"          - Image Index: {self.image_index.ntotal} 个向量")
             print(f"          - Mean Index: {self.mean_index.ntotal} 个向量")
        print("[Retriever] 初始化完成。")


    def retrieve(self, query: Union[str, Dict], k: int = 5) -> List[Dict]:
        """
        执行完整的检索流程：接收查询 -> 编码查询 -> 选择合适的索引和查询向量 ->
        在 Faiss 中搜索相似向量 -> 获取对应的文档元数据 -> 组合并返回结果。

        Args:
            query (Union[str, Dict]): 用户查询。可以是：
                - str: 一个纯文本查询字符串。
                - Dict: 一个字典，用于表示不同类型的查询：
                    - {'text': '...', 'image_path': '...'} : 多模态查询，结合文本和图像。
                    - {'image_path': '...'} : 纯图像查询，只使用图像内容。
                    - {'text': '...'} : 纯文本查询 (与直接传入字符串效果相同)。
                    字典中必须至少包含 'text' 或 'image_path' 之一。
            k (int): 指定希望检索的最相似文档的数量 (Top-K)。默认为 5。

        Returns:
            List[Dict]: 一个按相似度得分降序排列的文档列表。
                        列表中的每个字典代表一个检索到的文档，包含以下键：
                        - 'id': 原始文档 ID (str)。
                        - 'text': 文档文本内容 (str 或 None)。
                        - 'image_path': 关联图像路径 (str 或 None)。
                        - 'internal_id': 数据库和 Faiss 使用的内部 ID (int)。
                        - 'score': 该文档与查询的相似度得分 (float)。对于内积搜索，得分越高表示越相似。
                        如果查询无效、编码失败或搜索无结果，则返回空列表 `[]`。
        """
        print(f"\n[Retriever] 开始执行检索，目标是获取 Top-{k} 相关文档...")

        query_text = None        # 用于存储查询中的文本部分
        query_image_path = None  # 用于存储查询中的图像路径部分
        query_type = "unknown"   # 用于日志记录和后续逻辑判断的查询类型

        # --- 1. 解析查询输入，确定查询类型和内容 ---
        print("[Retriever]   - 步骤 1: 解析查询输入...")
        if isinstance(query, str):
            # 如果查询是字符串，则为纯文本查询
            query_text = query
            query_type = "纯文本"
            # 打印查询类型和部分内容（截断长文本）
            print(f"[Retriever]     查询类型确定为: {query_type}")
            print(f"[Retriever]     查询内容: '{query_text[:100]}{'...' if len(query_text)>100 else ''}'")
        elif isinstance(query, dict):
            # 如果查询是字典，从中提取文本和图像路径
            query_text = query.get('text')
            query_image_path = query.get('image_path')
            # 根据提取到的内容判断具体查询类型
            if query_text and query_image_path:
                query_type = "多模态"
                print(f"[Retriever]     查询类型确定为: {query_type}")
                print(f"[Retriever]     查询文本: '{query_text[:50]}{'...' if len(query_text)>50 else ''}'")
                print(f"[Retriever]     查询图像: '{os.path.basename(query_image_path)}'") # 只显示文件名
            elif query_image_path:
                query_type = "纯图像"
                print(f"[Retriever]     查询类型确定为: {query_type}")
                print(f"[Retriever]     查询图像: '{os.path.basename(query_image_path)}'")
            elif query_text:
                query_type = "纯文本" # 通过字典传入的纯文本查询
                print(f"[Retriever]     查询类型确定为: {query_type} (来自字典)")
                print(f"[Retriever]     查询内容: '{query_text[:100]}{'...' if len(query_text)>100 else ''}'")
            else:
                # 如果字典中既没有 'text' 也没有 'image_path'，是无效查询
                print("[Retriever] 错误: 查询字典无效，必须包含 'text' 或 'image_path' 键。")
                return [] # 返回空列表
        else:
            # 如果查询既不是字符串也不是字典，是不支持的类型
            print(f"[Retriever] 错误: 不支持的查询类型: {type(query)}。查询必须是 str 或 dict。")
            return [] # 返回空列表

        # --- 2. 使用 Encoder 对查询进行编码 ---
        # 无论查询类型如何，都尝试获取文本、图像和平均向量，后续根据类型选用
        print(f"[Retriever]   - 步骤 2: 使用 MultimodalEncoder 对查询进行编码...")
        try:
            # 调用 encoder 的 encode 方法
            encoded_query = self.encoder.encode(text=query_text, image_path=query_image_path)
            # 从返回的字典中获取各个向量
            query_text_vec = encoded_query.get('text_vector')
            query_image_vec = encoded_query.get('image_vector')
            query_mean_vec = encoded_query.get('mean_vector')
            # 检查是否至少有一个向量成功生成
            if query_text_vec is None and query_image_vec is None and query_mean_vec is None:
                 print("[Retriever] 警告: 查询编码未能生成任何有效的向量。无法继续检索。")
                 return []
            print("[Retriever]     查询编码完成。")
        except Exception as e:
            # 处理编码过程中可能发生的异常
            print(f"[Retriever] 错误: 编码查询时发生严重错误: {e}")
            return [] # 编码失败则无法检索，返回空列表

        # --- 3. 根据查询类型选择目标 Faiss 索引和查询向量 ---
        print(f"[Retriever]   - 步骤 3: 根据查询类型 '{query_type}' 选择搜索策略...")
        target_index = None   # 将用于搜索的 Faiss 索引
        query_vector = None   # 将用于搜索的查询向量
        index_name = "N/A"    # 用于日志记录的索引名称

        if query_type == "纯文本":
            # 对于纯文本查询，优先使用文本向量在文本索引中搜索
            if query_text_vec is not None and self.text_index.ntotal > 0:
                target_index = self.text_index
                query_vector = query_text_vec
                index_name = "Text Index"
                print(f"[Retriever]     策略: 使用文本查询向量在 Text Index (含 {self.text_index.ntotal} 向量) 中搜索。")
            else:
                 # 如果文本向量编码失败或文本索引为空，则无法执行
                 reason = "文本查询向量编码失败" if query_text_vec is None else f"Text Index 为空 ({self.text_index.ntotal} vectors)"
                 print(f"[Retriever] 警告: 无法执行纯文本查询，因为 {reason}。")
                 return []
        elif query_type == "纯图像":
            # 对于纯图像查询，优先使用图像向量在图像索引中搜索
            if query_image_vec is not None and self.image_index.ntotal > 0:
                target_index = self.image_index
                query_vector = query_image_vec
                index_name = "Image Index"
                print(f"[Retriever]     策略: 使用图像查询向量在 Image Index (含 {self.image_index.ntotal} 向量) 中搜索。")
            else:
                reason = "图像查询向量编码失败" if query_image_vec is None else f"Image Index 为空 ({self.image_index.ntotal} vectors)"
                print(f"[Retriever] 警告: 无法执行纯图像查询，因为 {reason}。")
                return []
        elif query_type == "多模态":
            # 对于多模态查询，优先使用平均向量在平均索引中搜索
            if query_mean_vec is not None and self.mean_index.ntotal > 0:
                target_index = self.mean_index
                query_vector = query_mean_vec
                index_name = "Mean Index"
                print(f"[Retriever]     策略: 使用平均查询向量在 Mean Index (含 {self.mean_index.ntotal} 向量) 中搜索。")
            # *** 回退策略 (Fallback Strategy) ***
            # 如果平均向量不可用 (编码失败) 或平均索引为空，尝试回退到使用文本向量在文本索引中搜索
            elif query_text_vec is not None and self.text_index.ntotal > 0:
                 print("[Retriever]     警告: Mean Index 或平均查询向量不可用。")
                 print("[Retriever]     回退策略: 改为使用文本查询向量在 Text Index (含 {self.text_index.ntotal} 向量) 中搜索。")
                 target_index = self.text_index
                 query_vector = query_text_vec
                 index_name = "Text Index (Fallback)"
            # 如果连回退策略也无法执行
            else:
                reason = ""
                if query_mean_vec is None and query_text_vec is None: reason += "平均和文本查询向量都编码失败; "
                if self.mean_index.ntotal == 0 and self.text_index.ntotal == 0: reason += f"Mean Index ({self.mean_index.ntotal}) 和 Text Index ({self.text_index.ntotal}) 都为空; "
                if not reason: reason = "未知原因导致多模态搜索无法进行" # 保险
                print(f"[Retriever] 警告: 无法执行多模态查询，因为 {reason.strip()}。")
                return []
        else: # query_type == "unknown" 或其他逻辑错误
             print("[Retriever] 错误: 无法确定有效的查询类型或找不到可用的查询向量/索引。")
             return []

        # --- 4. 执行 Faiss 相似度搜索 ---
        print(f"[Retriever]   - 步骤 4: 在选定的 '{index_name}' 中执行 Faiss Top-{k} 搜索...")
        try:
            # Faiss 的 `search` 方法需要查询向量是二维数组，形状为 (n_queries, dimension)
            # 这里我们只有一个查询，所以 n_queries=1
            # 使用 reshape(1, -1) 将一维向量转换为正确的形状
            query_vector_np = query_vector.reshape(1, self.vector_dimension)

            # 调用 Faiss 索引的 `search` 方法
            # D: 返回一个二维数组，包含每个查询结果的距离/得分。形状 (n_queries, k)。
            #    对于 IndexFlatIP (内积)，得分越高表示越相似。
            # I: 返回一个二维数组，包含每个查询结果的向量 ID (我们存的是 internal_id)。形状 (n_queries, k)。
            scores, internal_ids_np = target_index.search(query_vector_np, k)

            # 从返回结果中提取我们需要的 ID 列表和得分列表 (因为 n_queries=1，所以取索引 0)
            # Faiss 在找不到足够 k 个结果时，可能会用 -1 填充 ID 数组，需要过滤掉
            retrieved_internal_ids = []
            retrieved_scores = []
            for id_val, score_val in zip(internal_ids_np[0], scores[0]):
                if id_val != -1: # 只保留有效的 ID (非 -1)
                    retrieved_internal_ids.append(int(id_val)) # 确保 ID 是整数
                    retrieved_scores.append(float(score_val)) # 确保得分是浮点数

            # 检查是否找到了任何有效结果
            if not retrieved_internal_ids:
                print("[Retriever]     Faiss 搜索完成，但未返回任何有效的结果 ID (可能索引为空或没有足够相似的向量)。")
                return [] # 搜索无结果，返回空列表
            print(f"[Retriever]     Faiss 搜索完成，初步找到 {len(retrieved_internal_ids)} 个候选文档的 internal_id。")
            # 打印找到的 ID 和得分 (调试用，可选)
            # for i, (id_val, score_val) in enumerate(zip(retrieved_internal_ids, retrieved_scores)):
            #     print(f"        {i+1}. ID: {id_val}, Score: {score_val:.4f}")

        except Exception as e:
            # 处理 Faiss 搜索过程中可能发生的错误
            print(f"[Retriever] 错误: 执行 Faiss 搜索时发生错误: {e}")
            import traceback
            traceback.print_exc() # 打印详细错误信息
            return [] # 搜索失败，返回空列表

        # --- 5. 根据 internal_ids 从数据库批量获取文档元数据 ---
        print(f"[Retriever]   - 步骤 5: 使用找到的 {len(retrieved_internal_ids)} 个 internal_id 从 SQLite 数据库批量获取文档元数据...")
        # 调用 Indexer 的批量获取方法
        documents_map = self.indexer.get_documents_by_internal_ids(retrieved_internal_ids)
        # documents_map 是一个字典 {internal_id: doc_data_dict}
        print(f"[Retriever]     已成功从数据库获取了 {len(documents_map)} 条对应的文档记录。")

        # --- 6. 组合结果：将元数据与相似度得分结合，并保持 Faiss 的排序 ---
        print(f"[Retriever]   - 步骤 6: 组合文档元数据与相似度得分...")
        retrieved_docs = [] # 初始化最终结果列表
        # 遍历 Faiss 返回的 ID 和得分列表 (它们是按得分降序排列的)
        for internal_id, score in zip(retrieved_internal_ids, retrieved_scores):
            # 尝试从数据库查询结果 (documents_map) 中获取对应 internal_id 的文档数据
            doc_data = documents_map.get(internal_id)
            # 如果找到了文档数据
            if doc_data:
                # 将相似度得分 ('score') 添加到文档数据字典中
                doc_data['score'] = score
                # 将这个包含所有信息的字典添加到最终结果列表
                retrieved_docs.append(doc_data)
            else:
                # **数据不一致警告**: 如果 Faiss 返回了一个 ID，但在数据库中找不到对应的记录，
                # 这通常表示 Faiss 索引和数据库之间存在某种不同步。
                # 可能是数据库记录被删除但索引未更新，或者索引过程中出现了错误。
                print(f"[Retriever] 警告: 在数据库中未能找到 Faiss 返回的 internal_id: {internal_id}。")
                print(f"          这可能表示 Faiss 索引与数据库元数据存在不一致。跳过此条目。")

        # Faiss 对于 IndexFlatIP 的搜索结果默认按内积得分（相似度）降序排列，所以我们不需要再排序。
        print(f"[Retriever] 检索流程完成，最终返回 {len(retrieved_docs)} 个文档。")
        return retrieved_docs


    def close(self):
        """
        关闭 Retriever 实例时调用的清理方法。
        Retriever 本身通常没有需要显式关闭的资源 (因为它依赖于 Indexer)。
        这里只打印一条日志信息。
        """
        print("[Retriever] 关闭 Retriever...")
        # 通常无需执行特定操作
        print("[Retriever] Retriever 关闭完成。")


# --- 4. 生成器 (Generator) - 负责利用 LLM 生成最终答案 ---

class Generator:
    """
    Generator 类负责与大语言模型 (LLM) API 进行交互，以生成最终的答案。
    它接收用户的原始查询和由 Retriever 检索到的相关文档上下文列表，
    然后：
    1.  **构建提示 (Prompt)**: 将用户的查询和格式化的上下文信息组合成一个结构化的提示，
        该提示指导 LLM 如何根据提供的上下文来回答问题，并遵循特定规则（如仅基于上下文、处理不确定性等）。
    2.  **调用 LLM API**: 将构建好的提示发送给指定的 LLM API (这里使用 ZhipuAI 的 API)。
    3.  **处理响应**: 对 LLM 返回的原始文本进行基本的后处理（例如去除多余空格）。

    此类依赖于 `zhipuai` 库与 ZhipuAI API 通信，并需要一个有效的 API Key。
    API Key 优先从构造函数参数获取，其次从环境变量 `ZHIPUAI_API_KEY` 读取。
    """
    def __init__(self, api_key: Optional[str] = None, model_name: str = "glm-4-flash-250414"):
        """
        初始化 Generator 实例。

        Args:
            api_key (Optional[str]): ZhipuAI 的 API Key。如果提供，将优先使用这个 Key。
                                     如果为 None，则会尝试从环境变量 `ZHIPUAI_API_KEY` 读取。
            model_name (str): 要调用的 ZhipuAI 模型名称。例如 "glm-4-flash", "glm-4" 等。
                              不同的模型具有不同的能力、速度和成本。默认为 "glm-4-flash-250414"。

        Raises:
            ValueError: 如果 `api_key` 参数为 None 且在环境变量 `ZHIPUAI_API_KEY` 中也找不到 Key。
                        没有 API Key，无法与 ZhipuAI 服务通信。
            Exception: 如果 ZhipuAI 客户端初始化失败 (例如网络问题、库安装问题)。
        """
        print(f"[Generator] 初始化 Generator，准备使用 ZhipuAI 模型: {model_name}")
        # 决定最终使用的 API Key：优先使用参数传入的，否则从环境变量获取
        final_api_key = api_key if api_key else os.getenv("ZHIPUAI_API_KEY")

        # 检查是否成功获取到 API Key
        if not final_api_key:
            # 如果没有 API Key，抛出 ValueError
            error_message = "[Generator] 错误: ZhipuAI API Key 未提供。请执行以下操作之一：\n" \
                            "  1. 在初始化 Generator 时通过 'api_key' 参数传入。\n" \
                            "  2. 设置环境变量 'ZHIPUAI_API_KEY'。"
            raise ValueError(error_message)

        # 尝试使用获取到的 API Key 初始化 ZhipuAI 客户端
        try:
            # 创建 ZhipuAI 客户端实例
            self.client = zhipuai.ZhipuAI(api_key=final_api_key)
            # 存储要使用的模型名称
            self.model_name = model_name
            print(f"[Generator] ZhipuAI 客户端已成功初始化。")
        except Exception as e:
             # 处理客户端初始化过程中可能发生的异常
             print(f"[Generator] 错误: 初始化 ZhipuAI 客户端失败: {e}")
             print(f"[Generator] 请确认 API Key 是否有效，并且 'zhipuai' 库已正确安装 (pip install zhipuai)。")
             raise e # 重新抛出异常，因为 Generator 无法在没有客户端的情况下工作
        print("[Generator] Generator 初始化完成。")

    def generate(self, query: str, context: List[Dict]) -> str:
        """
        根据用户提供的原始查询和 Retriever 返回的文档上下文列表，调用 LLM 生成回答。

        Args:
            query (str): 用户提出的原始问题字符串。
            context (List[Dict]): 一个文档字典列表，由 Retriever 的 `retrieve` 方法返回。
                                 每个字典包含 'id', 'text', 'image_path', 'internal_id', 'score' 等信息。

        Returns:
            str: 由大语言模型生成并经过基本后处理的文本响应。
                 如果在调用 API 时发生错误，会返回一条包含错误信息的提示性字符串。
        """
        print(f"\n[Generator] 开始为查询生成最终响应...")
        print(f"[Generator]   接收到用户查询: '{query[:100]}{'...' if len(query)>100 else ''}'")
        print(f"[Generator]   使用 {len(context)} 个检索到的文档作为生成上下文。")

        # --- 1. 构建发送给 LLM 的 Prompt (消息列表) ---
        print("[Generator]   - 步骤 1: 构建 Prompt (包含系统指令和上下文)...")
        # 调用内部方法 _build_messages 来创建符合 ZhipuAI API 格式的消息列表
        messages = self._build_messages(query, context)

        # 可选：打印部分 Prompt 内容以供调试
        # print(f"[Generator]     Prompt System Message (部分): {messages[0]['content'][:300]}...")
        # print(f"[Generator]     Prompt User Message: {messages[1]['content']}")
        print("[Generator]     Prompt 构建完成。")

        # --- 2. 调用 ZhipuAI Chat Completions API ---
        print(f"[Generator]   - 步骤 2: 调用 ZhipuAI Chat API (模型: {self.model_name})...")
        llm_response = "抱歉，生成响应时遇到未知问题。" # 初始化默认错误响应
        try:
            # 使用 ZhipuAI 客户端的 chat.completions.create 方法发送请求
            response = self.client.chat.completions.create(
                model=self.model_name,   # 指定要使用的模型
                messages=messages,       # 传入构建好的消息列表
                temperature=0.7,         # 控制生成文本的随机性/创造性。0.7 是一个常用值，平衡创造性和一致性。
                                         # 值越低越确定性，越高越随机。
                max_tokens=1024,         # 限制模型生成的最大 token 数量，防止响应过长或失控。
                # 其他可选参数: top_p, stream=True (流式输出) 等
            )
            # 从 API 响应中提取生成的文本内容
            # response.choices[0] 是第一个（通常也是唯一一个）候选响应
            # .message.content 包含实际的文本
            llm_response = response.choices[0].message.content

            # 记录 token 使用情况 (用于成本估算或监控)
            completion_tokens = response.usage.completion_tokens # 生成的 token 数
            prompt_tokens = response.usage.prompt_tokens         # 输入提示的 token 数
            total_tokens = response.usage.total_tokens           # 总 token 数
            print(f"[Generator]     ZhipuAI API 调用成功。")
            print(f"              Token 使用情况 - 输入: {prompt_tokens}, 输出: {completion_tokens}, 总计: {total_tokens}")

        # --- 错误处理: 捕获 ZhipuAI 可能抛出的特定异常 ---
        except zhipuai.APIStatusError as e:
             # API 返回了非 2xx 的状态码 (例如 4xx 客户端错误, 5xx 服务器错误)
             print(f"[Generator]   - 错误: ZhipuAI API 返回状态错误。")
             print(f"              Status Code: {e.status_code}")
             print(f"              错误信息: {e.message}")
             # 可以考虑打印更详细的响应体 (e.response.text)，但可能包含敏感信息，谨慎使用
             # print(f"              详细响应: {e.response.text}")
             llm_response = f"抱歉，调用语言模型时遇到 API 错误 (状态码: {e.status_code})，请检查输入或稍后重试。"
        except zhipuai.APIConnectionError as e:
             # 无法连接到 ZhipuAI 服务器 (例如网络问题)
             print(f"[Generator]   - 错误: ZhipuAI API 连接失败: {e}")
             llm_response = f"抱歉，无法连接到语言模型服务。请检查网络连接或确认 API 端点是否正确。"
        except Exception as e:
             # 捕获其他在调用过程中可能发生的未知错误
             print(f"[Generator]   - 错误: 调用 LLM 时发生未预料的异常: {e}")
             import traceback
             traceback.print_exc() # 打印详细的堆栈跟踪
             llm_response = f"抱歉，生成响应时发生了意外的内部错误。"

        # --- 3. 对 LLM 的原始响应进行后处理 ---
        print("[Generator]   - 步骤 3: 对 LLM 响应进行后处理...")
        # 调用内部方法 _postprocess_response
        processed_response = self._postprocess_response(llm_response)
        print("[Generator]     响应后处理完成。")
        print("[Generator] 响应生成流程结束。")
        # 返回最终处理过的响应文本
        return processed_response

    def _build_messages(self, query: str, context: List[Dict]) -> List[Dict]:
        """
        根据用户查询和检索到的上下文文档，构建符合 ZhipuAI Chat API 要求的消息列表 (messages)。
        这个列表通常包含两条消息：
        1.  **System Message**: 提供系统级的指令，设定 LLM 的角色、行为规则，并注入检索到的上下文信息。
        2.  **User Message**: 包含用户原始的查询。

        Args:
            query (str): 用户的原始查询。
            context (List[Dict]): Retriever 返回的上下文文档列表。

        Returns:
            List[Dict]: 一个包含 'role' 和 'content' 的字典列表，可以直接传递给 ZhipuAI API。
        """
        # --- 构建 System Message ---
        # System Message 用于设定 LLM 的行为模式和约束
        system_message_content = """
        你是一个专业的、严谨的文档问答助手。请根据下面提供的 "参考文档" 部分中的信息来回答用户的问题。

        # 必须遵守的核心规则:
        1.  **严格依据**: 你的回答必须 **完全** 基于 "参考文档" 中提供的信息。严禁使用任何你在训练中获得的外部知识、个人观点或进行任何形式的猜测。
        2.  **处理不确定性**: 如果 "参考文档" 中的信息不足以回答用户的问题，或者问题与文档内容无关，你必须明确指出信息的缺乏。可以说：“根据提供的文档，我无法回答这个问题。”或“文档中没有包含足够的信息来回答关于...的问题。”
        3.  **关于图片**: 你无法直接“看到”图片。你对图片的理解 **只能** 来源于文档中对该图片的 **文本描述**，以及文档关联的 **图片文件名**（如果提供了的话）。不要声称你能看到图片内容。
        4.  **回答图片相关问题**:
            - 如果用户问及某张图片的内容，请在 "参考文档" 的文本描述中查找是否有相关的说明。
            - 如果找到了文本描述，请根据该文本描述来回答。
            - 如果只提供了图片文件名但没有文本描述，你可以提及这个文件名（例如，“文档提到了一个关联图片'circuit_diagram.png'”），并说明文档中缺少对该图片内容的具体文字描述。
            - 如果文档中既没有图片描述也没有文件名，或者问题与文档中的图片无关，请按照规则 2 处理。
        5.  **引用来源 (可选但推荐)**: 如果可能，请在回答中指明你的答案是基于哪些参考文档。例如：“根据文档 ID: 'BGREF_01' 的描述...” 或 “文档 1 (ID: XXX) 提到...”。
        6.  **回答风格**: 回答应尽可能简洁、清晰、直接。避免冗长和不必要的客套话。

        # 参考文档:
        --- 开始参考文档 ---
        """.strip() # .strip() 移除首尾多余空白

        # --- 格式化上下文信息 ---
        context_parts = [] # 用于存储格式化后的每个文档信息片段
        if not context:
            # 如果 Retriever 没有返回任何上下文文档
            context_parts.append("（注意：本次未能检索到与问题相关的文档。请基于此情况回答。）")
        else:
            # 遍历每个检索到的文档
            for i, doc in enumerate(context):
                # 提取文档信息，提供默认值以防键不存在
                doc_id = doc.get('id', '未知ID')
                internal_id = doc.get('internal_id', 'N/A') # 内部 ID，主要用于调试
                score = doc.get('score', 'N/A')             # 相似度得分
                text_content = doc.get('text', '无文本内容')
                image_path = doc.get('image_path')          # 图像路径

                # 格式化图像信息
                image_info = f"关联图片文件名: '{os.path.basename(image_path)}'" if image_path else "无关联图片信息"

                # 截断过长的文本内容，避免 Prompt 过长超出模型限制
                # 保留前 500 个字符，如果原文更长则添加省略号
                # 这个长度可以根据需要调整
                max_text_len = 500
                truncated_text = text_content[:max_text_len] + ('...' if len(text_content) > max_text_len else '')

                # 构建单个文档的格式化字符串
                context_parts.append(f"--- 文档 {i+1} ---")
                context_parts.append(f"  原始文档 ID: {doc_id}")
                # context_parts.append(f"  内部数据库 ID: {internal_id}") # 可以注释掉，LLM 不需要这个信息
                context_parts.append(f"  与查询的相关度得分: {score:.4f}" if isinstance(score, float) else f"  与查询的相关度得分: {score}")
                context_parts.append(f"  文本内容: {truncated_text}")
                context_parts.append(f"  {image_info}")
                # 添加分隔符，使多个文档更清晰（最后一个文档后不需要）
                # if i < len(context) - 1:
                #     context_parts.append("-" * 20) # 使用更明显的分割线

        # 将所有格式化的文档片段合并成一个字符串，用换行符连接
        context_section = "\n".join(context_parts)

        # 将格式化的上下文添加到 System Message 的末尾
        system_message_content += "\n" + context_section + "\n--- 结束参考文档 ---"

        # --- 构建最终的消息列表 ---
        messages = [
            {"role": "system", "content": system_message_content}, # 系统消息
            {"role": "user", "content": query}                     # 用户消息 (原始查询)
        ]
        return messages

    def _postprocess_response(self, llm_response: str) -> str:
        """
        对从 LLM API 获取的原始响应字符串进行基本的后处理。
        目前只执行简单的去除首尾空白字符操作。
        未来可以根据需要在这里添加更复杂的处理逻辑，例如：
        - 移除 LLM 可能添加的冗余前缀或后缀（如 "根据文档..." 如果我们不想要的话）。
        - 格式化输出（例如，将列表转换为 markdown 格式）。
        - 进行敏感信息过滤（如果需要）。
        - 修正语法或拼写错误（可以使用其他工具）。

        Args:
            llm_response (str): 从 LLM API 收到的原始文本响应。

        Returns:
            str: 经过后处理的文本响应。
        """
        # .strip() 移除字符串开头和结尾的任何空白字符 (空格, 制表符, 换行符等)
        processed_response = llm_response.strip()
        # 这里可以添加更多后处理步骤
        # processed_response = processed_response.replace("某些固定前缀", "")
        return processed_response

    def close(self):
        """
        关闭 Generator 实例时调用的清理方法。
        对于 ZhipuAI 的 Python 客户端，通常不需要显式关闭 HTTP 连接或资源。
        这里只打印一条日志信息。
        """
        print("[Generator] 关闭 Generator...")
        # ZhipuAI 客户端通常不需要显式关闭
        print("[Generator] Generator 关闭完成。")


# --- 示例使用流程 ---
# 当这个脚本被直接运行时 (__name__ == "__main__")，以下代码块将被执行。
# 它演示了如何将上面定义的各个组件 (Loader, Indexer, Retriever, Generator) 串联起来，
# 构建并运行一个完整的多模态 RAG 查询流程。
if __name__ == "__main__":
    print("\n" + "="*60)
    print("========= 多模态 RAG 系统 (增强版 - 分离向量索引) =========")
    print("=========         示例程序执行开始                =========")
    print("="*60 + "\n")

    # --- 配置参数 ---
    # 这些参数控制着系统的行为，如数据源、存储路径和使用的模型。
    # **数据源配置**
    JSON_DATA_PATH = 'data.json'       # 指定包含文档元数据 (ID, description) 的 JSON 文件路径。
    IMAGE_DIR_PATH = 'images'          # 指定存放与 JSON 数据对应的图像文件的目录路径。

    # **持久化存储配置** (数据库和 Faiss 索引文件)
    # 使用新文件名 (v2) 以区别于可能的旧版本文件。
    DB_FILE = 'multimodal_rag_data_v2.db' # SQLite 数据库文件路径，用于存储文档元数据。
    FAISS_TEXT_INDEX_FILE = 'text_index_v2.faiss'      # 存储文本向量的 Faiss 索引文件路径。
    FAISS_IMAGE_INDEX_FILE = 'image_index_v2.faiss'    # 存储图像向量的 Faiss 索引文件路径。
    FAISS_MEAN_INDEX_FILE = 'mean_index_v2.faiss'     # 存储平均向量 (文本+图像) 的 Faiss 索引文件路径。

    # **模型配置**
    CLIP_MODEL = "openai/clip-vit-base-patch32" # 指定用于文本和图像编码的 CLIP 模型。
                                                # 确保 Indexer 和 Retriever 使用相同的模型。
    LLM_MODEL = "glm-4-flash-250414"            # 指定用于生成最终答案的 ZhipuAI 大语言模型。
                                                # 可以替换为其他可用模型如 "glm-4"。

    # --- 清理旧文件 (可选步骤) ---
    # 为了确保每次运行都是从干净的状态开始（特别是用于测试或演示），
    # 可以选择在开始前删除旧的数据库和索引文件。
    print("--- [主流程] 准备环境：(可选) 清理旧的数据库和 Faiss 索引文件 ---")
    files_to_clean = [DB_FILE, FAISS_TEXT_INDEX_FILE, FAISS_IMAGE_INDEX_FILE, FAISS_MEAN_INDEX_FILE]
    for f_path in files_to_clean:
        if os.path.exists(f_path): # 检查文件是否存在
            try:
                os.remove(f_path) # 尝试删除文件
                print(f"  - 已成功清理旧文件: {f_path}")
            except OSError as e:
                 # 如果删除失败 (例如权限问题)，打印错误信息
                 print(f"  - 清理文件 {f_path} 时发生错误: {e}")
    print("--- [主流程] 环境清理完成 (如果文件存在)。---\n")
    time.sleep(1) # 短暂停顿，让输出更清晰

    # --- 1. 加载数据和关联图片 ---
    print("--- [主流程] 步骤 1: 从 JSON 加载文档数据并尝试关联图片 ---")
    # 调用数据加载函数
    documents_to_index = load_data_from_json_and_associate_images(JSON_DATA_PATH, IMAGE_DIR_PATH)
    # 检查是否成功加载到任何数据
    if not documents_to_index:
        print("\n[主流程] 严重错误：未能从 JSON 文件加载任何有效的文档数据。")
        print(f"          请检查 '{JSON_DATA_PATH}' 文件是否存在、格式是否正确，")
        print(f"          以及其中的记录是否包含 'name' 和 'description' 字段。")
        print("          程序将退出。")
        exit(1) # 退出程序，因为没有数据无法继续
    print("--- [主流程] 步骤 1 完成 ---\n")
    time.sleep(1)

    # --- 2. 初始化 Indexer 并建立索引 ---
    print("--- [主流程] 步骤 2: 初始化 Indexer 并对加载的文档建立索引 ---")
    indexer = None # 初始化 indexer 变量为 None
    try:
        # 创建 Indexer 实例，传入数据库和索引文件的路径，以及 CLIP 模型名称
        indexer = Indexer(
            db_path=DB_FILE,
            faiss_text_index_path=FAISS_TEXT_INDEX_FILE,
            faiss_image_index_path=FAISS_IMAGE_INDEX_FILE,
            faiss_mean_index_path=FAISS_MEAN_INDEX_FILE,
            clip_model_name=CLIP_MODEL
        )
        # 调用 Indexer 的 index_documents 方法，传入加载好的文档列表
        # 这个过程会进行编码、存数据库、添加到 Faiss 索引
        indexer.index_documents(documents_to_index)

        # 索引建立完成后，打印状态信息进行核对
        print("\n[主流程] 索引建立完成后的状态检查:")
        text_count = indexer.text_index.ntotal   # 获取文本索引中的向量数
        image_count = indexer.image_index.ntotal # 获取图像索引中的向量数
        mean_count = indexer.mean_index.ntotal  # 获取平均索引中的向量数
        db_doc_count = indexer.get_document_count() # 获取数据库中的记录数
        print(f"  - SQLite 数据库 ('{DB_FILE}') 中文档记录数: {db_doc_count}")
        print(f"  - Text Faiss Index ('{FAISS_TEXT_INDEX_FILE}') 中向量数: {text_count}")
        print(f"  - Image Faiss Index ('{FAISS_IMAGE_INDEX_FILE}') 中向量数: {image_count}")
        print(f"  - Mean Faiss Index ('{FAISS_MEAN_INDEX_FILE}') 中向量数: {mean_count}")

        # 检查是否存在所有索引都为空的情况
        if text_count == 0 and image_count == 0 and mean_count == 0 and db_doc_count > 0:
             print("[主流程] 警告：数据库中有记录，但所有 Faiss 索引都为空！")
             print("          这可能意味着文档编码过程全部失败。后续的检索操作将无法返回任何结果。")
             print("          请检查 Indexer 的日志输出以诊断编码问题。")
        elif db_doc_count == 0:
             print("[主流程] 警告：数据库和所有 Faiss 索引都为空。可能是初始数据为空或处理过程中所有条目都被跳过。")

    except Exception as e:
         # 捕获 Indexer 初始化或索引建立过程中可能发生的严重错误
         print(f"\n[主流程] 严重错误：在初始化 Indexer 或建立索引的过程中发生异常: {e}")
         # 打印详细的错误堆栈信息，帮助调试
         import traceback
         traceback.print_exc()
         print("          由于 Indexer 未能成功准备，后续步骤可能无法进行。")
         indexer = None # 将 indexer 设为 None，表示初始化失败

    print("--- [主流程] 步骤 2 完成 ---\n")
    time.sleep(1)

    # --- 3. 初始化 Retriever ---
    print("--- [主流程] 步骤 3: 初始化 Retriever ---")
    retriever = None # 初始化 retriever 变量为 None
    # 只有在 Indexer 成功初始化 (indexer is not None)
    # 并且至少有一个 Faiss 索引包含向量 (ntotal > 0) 时，才尝试初始化 Retriever
    # 因为一个没有任何可搜索向量的 Retriever 是无用的。
    if indexer and (indexer.text_index.ntotal > 0 or indexer.image_index.ntotal > 0 or indexer.mean_index.ntotal > 0):
        try:
            # 创建 Retriever 实例，传入已经准备好的 Indexer
            retriever = Retriever(indexer=indexer)
        except Exception as e:
             # 捕获 Retriever 初始化过程中可能发生的错误
             print(f"\n[主流程] 错误：初始化 Retriever 时发生异常: {e}")
             import traceback
             traceback.print_exc()
             retriever = None # 初始化失败
    else:
         # 如果 Indexer 初始化失败或所有索引都为空
         print("[主流程] 跳过 Retriever 初始化，因为 Indexer 未成功初始化或所有 Faiss 索引均为空。")
         if indexer is None:
             print("          (Indexer 初始化失败，请检查步骤 2 的日志)")
         elif indexer:
             print("          (Indexer 初始化成功，但所有索引均为空，可能是编码问题或数据问题，请检查步骤 2 日志)")

    print("--- [主流程] 步骤 3 完成 ---\n")
    time.sleep(1)

    # --- 4. 初始化 Generator ---
    print("--- [主流程] 步骤 4: 初始化 Generator (需要 ZHIPUAI_API_KEY) ---")
    generator = None # 初始化 generator 变量为 None
    # 检查 ZhipuAI API Key 是否已设置在环境变量中
    zhipuai_api_key_present = bool(os.getenv("ZHIPUAI_API_KEY"))
    if not zhipuai_api_key_present:
        # 如果环境变量中没有找到 API Key
        print("[主流程] 警告: 未找到环境变量 'ZHIPUAI_API_KEY'。")
        print("          Generator 将无法与 ZhipuAI API 通信，生成步骤将被跳过。")
        print("          要启用生成功能，请先设置环境变量，例如在 Linux/macOS 中运行:")
        print("          export ZHIPUAI_API_KEY='你的有效API Key'")
        print("          然后在同一个终端会话中重新运行此脚本。")
    else:
        # 如果找到了 API Key，尝试初始化 Generator
        try:
            # 创建 Generator 实例，它会自动从环境变量读取 API Key
            # 同时指定要使用的 LLM 模型
            generator = Generator(model_name=LLM_MODEL)
        except Exception as e:
             # 捕获 Generator 初始化过程中可能发生的错误 (例如 API Key 无效、网络问题)
             print(f"\n[主流程] 错误：初始化 Generator 时发生异常: {e}")
             import traceback
             traceback.print_exc()
             generator = None # 初始化失败

    print("--- [主流程] 步骤 4 完成 ---\n")
    time.sleep(1)

    # --- 5. 执行查询示例 ---
    print("--- [主流程] 步骤 5: 执行 RAG 查询示例 (检索 + 生成) ---")

    # 只有当 Retriever 和 Generator 都成功初始化后，才执行查询流程
    if retriever and generator:
        print("[主流程] Retriever 和 Generator 都已成功初始化，开始执行查询示例...")

        # --- 辅助函数：用于格式化打印检索到的文档列表 ---
        def print_retrieved_docs(docs: List[Dict]):
            """简洁地打印检索结果列表的关键信息。"""
            if not docs:
                print("    >> 检索结果：未找到相关文档。")
                return
            print(f"    >> 检索结果：找到 Top-{len(docs)} 相关文档:")
            # 遍历每个检索到的文档字典
            for i, doc in enumerate(docs):
                score = doc.get('score', 'N/A') # 获取相似度得分
                # 格式化得分，保留4位小数
                score_str = f"{score:.4f}" if isinstance(score, float) else str(score)
                # 获取文本内容，并截断以保持简洁
                text_preview = doc.get('text', '无文本内容')[:80] # 取前80个字符
                if len(doc.get('text', '')) > 80: text_preview += "..."
                # 检查是否有图像路径，如果有，则添加图像文件名信息
                img_info = f", 关联图片: '{os.path.basename(doc['image_path'])}'" if doc.get('image_path') else ""
                # 打印格式化的信息
                print(f"      {i+1}. ID: {doc.get('id', 'N/A')} (Score: {score_str})")
                print(f"         Text: '{text_preview}'{img_info}")
            print("    " + "-"*40) # 打印分隔线

        # --- 准备示例查询数据 ---
        # **纯文本查询示例**
        text_queries = [
            "什么是带隙基准电压源 (bandgap voltage reference)？",
            "请解释 PTAT 电流是如何产生的，以及它的作用。",
            "描述一下使用 BJT（双极结型晶体管）实现的带隙基准电路的基本结构。",
            "提高带隙基准电压源精度的常用方法有哪些？",
            "什么是曲率补偿 (curvature compensation)？", # 可能会找不到相关文档
        ]

        # **图像和多模态查询示例**
        # 首先，从加载的文档中找出那些确实有关联图像文件的文档
        image_docs_available = []
        if documents_to_index: # 确保原始数据已加载
            for doc in documents_to_index:
                img_path = doc.get('image_path')
                # 检查 image_path 是否存在，并且对应的文件确实在磁盘上
                if img_path and os.path.exists(img_path):
                    image_docs_available.append({'id': doc['id'], 'image_path': img_path})

        image_queries_data = [] # 用于存储纯图像查询的数据
        multimodal_queries_data = [] # 用于存储多模态查询的数据

        if image_docs_available:
            # 如果找到了带有效图片的文档，从中随机选择几个作为示例
            num_samples = min(3, len(image_docs_available)) # 最多选3个，或实际可用数量
            print(f"[主流程] 发现了 {len(image_docs_available)} 个带有效图片的文档，将从中随机选择 {num_samples} 个用于图像/多模态查询示例。")
            selected_image_docs = random.sample(image_docs_available, num_samples)

            # 为每个选中的带图片文档创建查询数据
            for doc_info in selected_image_docs:
                doc_id = doc_info['id']
                img_path = doc_info['image_path']
                img_filename = os.path.basename(img_path) # 获取图像文件名

                # 1. 创建纯图像查询 (Image Query)
                #    - `query_input`: 提供给 Retriever 的输入，只包含图像路径。
                #    - `query_for_generator`: 提供给 Generator 的问题文本。这个问题是关于图像内容的，
                #                           但提示 LLM 要基于 *文档文本* 来回答（因为 LLM 看不到图）。
                #    - `description`: 用于打印日志，描述这个查询是关于什么的。
                image_queries_data.append({
                    'query_input': {'image_path': img_path},
                    'query_for_generator': f"这张图片（文件名: {img_filename}）展示了什么电路或概念？请根据相关文档的文本描述来回答。",
                    'description': f"纯图像查询：关于图片 '{img_filename}' 的内容"
                })

                # 2. 创建多模态查询 (Multimodal Query)
                #    - `query_input`: 提供给 Retriever 的输入，包含文本和图像路径。
                #                   文本部分通常是对图像的提问或结合图像的提问。
                #    - `query_for_generator`: 提供给 Generator 的问题文本，与给 Retriever 的文本部分相同。
                #    - `description`: 日志描述。
                multimodal_queries_data.append({
                    'query_input': {'text': f"请结合文档内容和这张图片 (文件名: {img_filename})，解释图中所示电路的工作原理或关键特性。", 'image_path': img_path},
                    'query_for_generator': f"请结合文档内容和这张图片 (文件名: {img_filename})，解释图中所示电路的工作原理或关键特性。",
                    'description': f"多模态查询：解释图片 '{img_filename}' 中电路的原理 (结合文本)"
                })
        else:
             # 如果没有找到任何带有效图片的文档
             print("[主流程] 警告: 在加载的数据中未找到任何有效的、且实际存在的图像文件。")
             print("          将跳过后续的纯图像查询和多模态查询示例。")


        # --- 组织所有查询并按类型执行 ---
        # 将所有准备好的查询按类型分组
        all_queries = [
            ("纯文本查询", text_queries),
            ("纯图像查询", image_queries_data),
            ("多模态查询", multimodal_queries_data)
        ]

        query_counter = 0 # 用于给所有查询进行编号
        # 遍历每个查询组 (纯文本, 纯图像, 多模态)
        for query_group_name, queries in all_queries:
            print("\n" + "#"*50)
            print(f">>> 开始执行 [{query_group_name}] 类型的查询示例 ({len(queries)} 个查询) <<<")
            print("#"*50 + "\n")

            # 如果当前类型的查询列表为空 (例如没有找到图片文档导致图像/多模态查询列表为空)
            if not queries:
                print(f"    (跳过 [{query_group_name}] 类型查询，因为没有准备好的查询数据)")
                continue # 跳到下一个查询类型

            # 遍历当前类型中的每个具体查询
            for i, query_item in enumerate(queries):
                query_counter += 1
                print(f"\n--- 执行查询 {query_counter} ({query_group_name} - {i+1}/{len(queries)}) ---")

                # 初始化变量，用于存储传递给 Retriever 和 Generator 的具体内容
                query_input_for_retriever = None # 给 Retriever 的输入 (str 或 dict)
                query_text_for_generator = None # 给 Generator 的问题文本 (str)
                query_description = None        # 用于日志输出的查询描述 (str)

                # 根据查询类型，解析 query_item 并设置上述变量
                if query_group_name == "纯文本查询":
                    # 纯文本查询比较简单，item 本身就是查询字符串
                    query_input_for_retriever = query_item
                    query_text_for_generator = query_item
                    query_description = query_item
                    print(f"用户查询 (文本): {query_description}")
                else: # 处理图像或多模态查询 (query_item 是一个字典)
                    query_input_for_retriever = query_item['query_input']
                    query_text_for_generator = query_item['query_for_generator']
                    query_description = query_item['description']
                    print(f"用户查询类型: {query_group_name}")
                    print(f"查询描述: {query_description}")
                    # 打印查询的具体输入内容
                    if 'text' in query_input_for_retriever:
                        print(f"  -> Retriever 输入 (文本部分): '{query_input_for_retriever['text'][:100]}...'")
                    if 'image_path' in query_input_for_retriever:
                        print(f"  -> Retriever 输入 (图像部分): '{os.path.basename(query_input_for_retriever['image_path'])}'")
                    print(f"  -> Generator 输入 (问题文本): '{query_text_for_generator[:100]}...'")

                print("-" * 20) # 分隔符

                retrieved_context = [] # 初始化检索到的上下文列表
                try:
                    # === **步骤 A: 执行检索 (Retrieve)** ===
                    print("  [检索阶段] 调用 Retriever.retrieve()...")
                    # 调用 retriever 的 retrieve 方法，传入查询输入和期望返回的数量 k
                    # k=5 表示希望找到最相关的 5 个文档
                    retrieved_context = retriever.retrieve(query_input_for_retriever, k=5)
                    # 使用辅助函数打印检索结果的摘要信息
                    print_retrieved_docs(retrieved_context)

                    # === **步骤 B: 执行生成 (Generate)** ===
                    # 只有在成功检索到至少一个文档时，才进行生成步骤
                    if retrieved_context:
                        print("\n  [生成阶段] 调用 Generator.generate() (使用检索到的上下文)...")
                        # 调用 generator 的 generate 方法，传入原始的用户问题文本和检索到的上下文列表
                        generated_response = generator.generate(query_text_for_generator, retrieved_context)

                        # 打印 LLM 生成的最终响应
                        print("\n  <<< LLM 生成的最终响应 >>>")
                        print("-" * 25)
                        print(generated_response)
                        print("-" * 25)
                    else:
                         # 如果没有检索到任何上下文
                         print("\n  [生成阶段] 跳过生成步骤，因为没有从 Retriever 获取到相关上下文。")
                         print("             (这可能是因为查询与所有文档都不够相似，或者索引为空)")

                except Exception as e:
                     # 捕获在处理单个查询（检索或生成阶段）时可能发生的错误
                     print(f"\n[主流程] 严重错误：处理查询 '{query_description}' 时发生异常: {e}")
                     import traceback
                     traceback.print_exc() # 打印详细错误信息

                print(f"\n--- 查询 {query_counter} 处理结束 ---")
                # 在两次查询之间添加一个较长的延时，原因：
                # 1. 方便用户在控制台查看每次查询的完整输出。
                # 2. 如果 LLM API 有速率限制，可以避免触发限制。
                if i < len(queries) - 1: # 只在查询之间加延时，最后一个查询后不加
                    delay_seconds = 5
                    print(f"\n... 等待 {delay_seconds} 秒后执行下一次查询 ...\n" + "-"*60 + "\n")
                    time.sleep(delay_seconds)

            # 当前查询类型的所有查询都已执行完毕
            print("\n" + "#"*50)
            print(f">>> [{query_group_name}] 类型查询示例结束 <<<")
            print("#"*50 + "\n")
            time.sleep(2) # 在不同查询类型之间也稍作停顿


    else:
        # 如果 Retriever 或 Generator 未能成功初始化
        print("\n[主流程] 跳过 RAG 查询示例执行步骤，因为以下一个或多个关键组件未能成功初始化：")
        if not retriever:
            print("  - Retriever 未初始化。")
            print("    (请检查步骤 2 [Indexer 初始化和索引建立] 和步骤 3 [Retriever 初始化] 的日志输出)")
        if not generator:
            print("  - Generator 未初始化。")
            print("    (请检查步骤 4 [Generator 初始化] 的日志输出，特别是关于 ZHIPUAI_API_KEY 的检查)")

    print("--- [主流程] 步骤 5 完成 ---\n")

    # --- 6. 清理资源 ---
    # 在程序结束前，调用各个组件的 close 方法，执行必要的清理工作（主要是保存 Faiss 索引）。
    print("--- [主流程] 步骤 6: 清理和关闭资源 ---")
    if retriever: # 如果 retriever 成功初始化
        retriever.close()
    if generator: # 如果 generator 成功初始化
        generator.close()
    if indexer:   # 如果 indexer 成功初始化
        # Indexer 的 close 方法会负责调用 save_indices 来保存 Faiss 文件
        indexer.close()
    print("--- [主流程] 资源清理完成。---\n")

    # --- 程序结束 ---
    print("\n" + "="*60)
    print("========= 多模态 RAG 系统示例程序执行完毕 =========")
    print("生成/更新的文件:")
    print(f"  - SQLite 数据库: '{DB_FILE}'")
    print(f"  - Faiss 文本向量索引: '{FAISS_TEXT_INDEX_FILE}'")
    print(f"  - Faiss 图像向量索引: '{FAISS_IMAGE_INDEX_FILE}'")
    print(f"  - Faiss 平均向量索引: '{FAISS_MEAN_INDEX_FILE}'")
    print("="*60 + "\n")